In [ ]:
# Open in colab, thats a better way to run the code 
# https://colab.research.google.com/drive/189HnSEgWebOB1I-KOUanbsUe5XQcZJsh?usp=sharing

In [1]:
#There is readymade data below, run the last section if running entire codebase is as issue
#configure mongoDb uri and make a database "Scrapping" and collections "topnews","insights","stockdata","layoffs" in it

#Installations

In [ ]:
!pip install transformers

In [ ]:
!pip install "pymongo[srv]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.9 MB/s eta 0:00:00


In [ ]:
!pip install yfinance

In [ ]:
pip install --upgrade yfinance pandas_datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61


In [ ]:
!pip install newsapi-python

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.1 MB/s eta 0:00:00


#fetch_from_google_cse(query) function

In [ ]:
GOOGLE_API_KEY="Ur Key"
GOOGLE_CX="Ur Key" #cnbc.com reuters.com techcrunch.com

In [ ]:
from datetime import datetime

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return datetime.min

In [ ]:
def fetch_from_google_cse(query,start=1):
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CX,
        "start":start
    }
    try:
        response = requests.get("https://www.googleapis.com/customsearch/v1", params=params)
        return response.json().get("items", [])
    except Exception as e:
        return f"Error fetching data from Google Custom Search API: {e}"

#GroqResponse(prompt) function

In [ ]:
from groq import Groq
import os
import json
import time
import re

client = Groq(api_key="Ur_key")

In [ ]:
def groq_response(prompt,need_json=False):
  try:
    if need_json:
     response = client.chat.completions.create(
         model="llama3-8b-8192",
         messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"},
         temperature=0.1,
         max_tokens=256
     )
    else:
     response = client.chat.completions.create(
          model="llama3-8b-8192",
          messages=[{"role": "user", "content": prompt}],
          temperature=0.1,
          max_tokens=256
      )

    text = response.choices[0].message.content
    return text
  except Exception as e:
        print(f"Groq API error: {e}")
        return ""

#Summarize(link) function

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import spacy

In [ ]:
def summarize(link):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9"
        }

        # Improved content extraction
        r = requests.get(link, headers=headers, timeout=15)
        soup = BeautifulSoup(r.text, 'html.parser')

        # Extract text with semantic hierarchy
        results = []
        for tag in ['h1', 'h2', 'h3', 'h4', 'p', 'ul', 'ol']:
            results += soup.find_all(tag)

        text = ' '.join([element.get_text(separator=' ', strip=True) for element in results])

        text = re.sub(r'\s+', ' ', text).strip()

        # Enhanced content validation
        if len(text.split()) < 80:  # Increased minimum threshold
            return "Insufficient meaningful content for summarization"

        # Context-aware truncation
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)

        # Get the first num_sentences sentences
        sentences = [sent.text.strip() for sent in doc.sents][:20]

        # Structured prompt for financial/news content
        prompt = f"""Generate a concise news summary focusing on:
          - Key entities (companies, people, figures)
          - Financial impacts and numbers

          Article content:
          {text[:3000]}  # Increased context window

          Summary (1 strictly sentence, journalistic tone):"""

        summary = groq_response(prompt)
        return summary

    except Exception as e:
        print(f"Error summarizing {link}: {str(e)[:200]}")
        return "Summary unavailable. The article may be paywalled or structured unusually."


#calculate_liquidity_change(article) function

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta

In [ ]:
company_to_ticker = {
    'meta': 'META',
    'amazon': 'AMZN',
    'microsoft': 'MSFT',
    'google': 'GOOGL',
    'apple': 'AAPL'
}

In [ ]:
def calculate_liquidity_change(article, window_days=3):
    print(f"\nArticle: {article.get('title')}")

    if not article.get('date') or not article.get('ticker'):
        print("Missing date/ticker")
        return None

    try:
        # Convert to timezone-aware datetime
        layoff_date = pd.to_datetime(article['date']).tz_localize('UTC')

        # Calculate date range with buffer
        start = layoff_date - timedelta(days=window_days+7)
        end = layoff_date + timedelta(days=window_days+7)

        # Fetch historical data
        lower_ticker = article['ticker'].lower()  # <-- Key fix: lowercase once
        ticker = company_to_ticker.get(lower_ticker)

        if not ticker:
            print(f"No ticker mapping for {article['ticker']}")
            return None

        # Use lowercase key to access stock_map
        ticker_symbol = company_to_ticker[lower_ticker]  # Now uses lowercase key

        stock = yf.Ticker(ticker_symbol)
        hist = stock.history(
            start=start.strftime('%Y-%m-%d'),
            end=end.strftime('%Y-%m-%d'),
            interval='1d'
        )

        if hist.empty:
            print(f"No data for {ticker_symbol} between {start.date()} - {end.date()}")
            return None

        # Convert hist index to UTC
        hist.index = hist.index.tz_convert('UTC')

        # Find nearest trading day
        time_diff = abs(hist.index - layoff_date)
        nearest_date = hist.index[time_diff.argmin()]
        print(f"Original: {layoff_date.date()} | Nearest trading day: {nearest_date.date()}")

        # Get pre/post windows
        pre_dates = hist.index[hist.index < nearest_date][-window_days:]
        post_dates = hist.index[hist.index > nearest_date][:window_days]

        if len(pre_dates) < 2 or len(post_dates) < 2:
            print(f"Insufficient data: Pre={len(pre_dates)}, Post={len(post_dates)}")
            return None

        # Calculate metrics
        pre_vol = hist.loc[pre_dates, 'Volume'].mean()
        post_vol = hist.loc[post_dates, 'Volume'].mean()
        vol_change = ((post_vol - pre_vol) / pre_vol) * 100

        pre_spread = (hist.loc[pre_dates, 'High'] - hist.loc[pre_dates, 'Low']).mean()
        post_spread = (hist.loc[post_dates, 'High'] - hist.loc[post_dates, 'Low']).mean()
        spread_change = ((post_spread - pre_spread) / pre_spread) * 100

        liquidity_change = (vol_change + spread_change) / 2

        x= round(liquidity_change, 1)
        print(f"Liquidity change: {liquidity_change:.1f}%")
        return x


    except Exception as e:
        print(f"Error: {str(e)}")
        return None


#sentiment(text) function

In [ ]:
from transformers import pipeline

In [ ]:
wrapper = pipeline("sentiment-analysis")
def sentiment_analysis(text):
    x=wrapper(text)
    ans=x[0]['score']
    if x[0]['label']=='NEGATIVE':
      ans=-x[0]['score']
    return ans

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


#TopNews

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
tickers=['meta','google','microsoft','apple','amazon']

In [ ]:
def extract_ticker(article):
    summary = article['snippet']
    doc = nlp(summary)
    companies = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    if not companies:
      title=article['title']
      doc = nlp(title)
      companies = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    for ticker in tickers:
      for i in companies:
        if ticker in i:
          return ticker
    return companies[0] if companies else None

In [ ]:
q = "top tech news from meta,amazon,microsoft,google or apple"
google_results = fetch_from_google_cse(q)

In [ ]:
topnews = []
seen_links = set()
for item in google_results:
    link = item['link']
    if link not in seen_links:
        date = None
        try:
            metatags = item.get('pagemap', {}).get('metatags', [{}])
            date = metatags[0].get('article:published_time') if metatags else None
        except (KeyError, IndexError):
            pass
        topnews.append({
            "ticker": extract_ticker(item),
            "title": item['title'].split('|')[0],
            "url": link,
            "summary": summarize(link),
            "date": date[:10] if date else None # Added date field
        })
        seen_links.add(link)
topnews = sorted(
topnews,
key=lambda x: parse_date(x.get('date')),
reverse=True
)

In [ ]:
for item in topnews:
  item['sentiment']=sentiment_analysis(item['summary'])
  item['liquidity']=calculate_liquidity_change(item)
  print(item['liquidity'])


Article: Apple, Meta fined as EU presses ahead with tech probes 
Original: 2025-04-23 | Nearest trading day: 2025-04-23
Liquidity change: 2.9%
2.9

Article: The biggest questions facing Big Tech ahead of earnings
Original: 2025-04-21 | Nearest trading day: 2025-04-21
Liquidity change: 9.3%
9.3

Article: Tech megacaps to spend more than $300 billion in 2025 to win in AI
Original: 2025-02-08 | Nearest trading day: 2025-02-07
Liquidity change: -15.6%
-15.6

Article: Cramer's week ahead: Earnings from Alphabet, Meta, Apple ...
No ticker mapping for CNBC
None

Article: Big Tech farms out AI power build, keeps the risk 
Original: 2024-10-03 | Nearest trading day: 2024-10-03
Liquidity change: -1.1%
-1.1

Article: Early sign-ups to EU's AI Pact include Amazon, Google, Microsoft ...
No ticker mapping for TechCrunch Daily News
None

Article: AI engineers face burnout in 'rat race' to stay competitive hits tech
Original: 2024-05-03 | Nearest trading day: 2024-05-03
Liquidity change: -11.9%
-11.9

In [ ]:
topnews

[{'ticker': 'Microsoft',
  'title': 'Apple, Meta fined as EU presses ahead with tech probes ',
  'url': 'https://www.reuters.com/sustainability/boards-policy-regulation/apple-fined-570-million-meta-228-million-breaching-eu-law-2025-04-23/',
  'summary': "Here is a concise news summary:\n\nThe UK's competition watchdog has launched a probe into the $13.25 billion merger between advertising giants Omnicom and Interpublic, to assess whether the deal could lead to a reduction in competition in the UK market, as Apple and Meta face fines for breaching EU antitrust rules.",
  'date': '2025-04-23',
  'sentiment': 0.833164393901825,
  'liquidity': np.float64(2.9)},
 {'ticker': 'Meta',
  'title': 'The biggest questions facing Big Tech ahead of earnings',
  'url': 'https://www.cnbc.com/2025/04/21/tech-earnings-trump-tariffs-tesla-apple-alphabet-meta-microsoft-amazon-nvidia.html',
  'summary': "As tech's megacap companies enter first-quarter earnings season, uncertainty reigns supreme due to Pres

#Layoffs

In [ ]:
def layoff_query(ticker):
    return f"{ticker} (layoff OR layoffs OR fired OR job cuts OR workforce reduction OR hiring OR hires OR employed OR recruitment OR expansion)"

In [ ]:
from datetime import datetime

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return datetime.min

In [ ]:
tickers=['meta','google','microsoft','apple','amazon']

In [ ]:
all_results = {}
seen_links = set()
for ticker in tickers:
    q = layoff_query(ticker)
    google_results = fetch_from_google_cse(q)
    ticker_results=[]
    for item in google_results:
        link = item['link']
        if link not in seen_links:
            date = None
            x=summarize(link)

            try:
                metatags = item.get('pagemap', {}).get('metatags', [{}])
                date = metatags[0].get('article:published_time') if metatags else None
            except (KeyError, IndexError):
                pass

            ticker_results.append({
                "ticker": ticker,
                "title": item['title'],
                "url": link,
                "summary": summarize(link),
                "date": date[:10] if date else None # Added date field
            })
            seen_links.add(link)
    sorted_results = sorted(
    ticker_results,
    key=lambda x: parse_date(x.get('date')),
    reverse=True
    )
    all_results[ticker]=sorted_results


In [ ]:
all_results

{'meta': [{'ticker': 'meta',
   'title': 'A comprehensive list of 2025 tech layoffs | TechCrunch',
   'url': 'https://techcrunch.com/2025/05/21/tech-layoffs-2025-list/',
   'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from companies such as Playtika, Microsoft, Chegg, and Meta, resulting in a total of 16,084 job losses in February alone, with the year's total layoffs surpassing 150,000 across 549 companies, according to independent tracker Layoffs.fyi.",
   'date': '2025-05-21'},
  {'ticker': 'meta',
   'title': 'Meta cuts jobs at its Reality Labs division | Reuters',
   'url': 'https://www.reuters.com/business/world-at-work/meta-cuts-jobs-its-reality-labs-division-2025-04-24/',
   'summary': "Here is a concise news summary:\n\nMeta, the parent company of Facebook and Instagram, has cut jobs at its Reality Labs division, a unit focuse

In [ ]:
def scrape_and_process(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all('p')
    results+=soup.find_all('span')
    text = [res.text for res in results]
    words = ' '.join(text).split(' ')[:350]
    ARTICLE = ' '.join(words)
    return ARTICLE

In [ ]:
def prompt_template(title,content):
  return (
                "From the article below, extract:\n"
                f"1. The main layoff number strictly a number and strictly related to {ticker} else return  (e.g. '10,000', if percentage, find out a number by multiplying that the timely workforce no;).\n"
                "2. The main reason for the layoffs (as a short phrase or sentence).\n"
                "Return a JSON object with two fields: 'layoffs' and 'reason'.\n\n"
                f"Title: {title}\n"
                f"Content: {content}\n"
            )

In [ ]:
def process_layoffs_and_reason(all_results):
    for ticker in all_results:
        print(f"\nProcessing {ticker} articles")
        for idx, article in enumerate(all_results[ticker]):
            print(f"Article {idx+1}/{len(all_results[ticker])}: {article.get('title','')}")
            prompt=prompt_template(article['title'],article['summary'])
            try:
                text = groq_response(prompt,need_json=True)
                print("Raw response:", text[:150] + "..." if len(text) > 150 else text)
                match = re.search(r'\{.*\}', text, re.DOTALL)
                if match:
                    data = json.loads(match.group(0))
                    article['layoffs'] = data.get('layoffs', '')
                    article['reason'] = data.get('reason', '')
                    print(f"Extracted: layoffs={article['layoffs']}, reason={article['reason']}")
                else:
                    print("No valid JSON found in response.")
                    article['layoffs'] = 'None'
                    article['reason'] = 'None'
            except Exception as e:
                print(f"Error: {e}")
                article['layoffs'] = 'None'
                article['reason'] = 'None'
            time.sleep(1)
    return all_results
final_results = process_layoffs_and_reason(all_results)




Processing meta articles
Article 1/10: A comprehensive list of 2025 tech layoffs | TechCrunch
Raw response: {
   "layoffs": "150,000",
   "reason": "significant wave of layoffs in 2025"
}
Extracted: layoffs=150,000, reason=significant wave of layoffs in 2025
Article 2/10: Meta cuts jobs at its Reality Labs division | Reuters
Raw response: {
   "layoffs": "10,000",
   "reason": "significant reduction in workforce"
}
Extracted: layoffs=10,000, reason=significant reduction in workforce
Article 3/10: Meta prepares for layoffs on Monday | Reuters
Raw response: {
   "layoffs": "10,000",
   "reason": "no specific financial numbers disclosed"
}
Extracted: layoffs=10,000, reason=no specific financial numbers disclosed
Article 4/10: Meta announces 5% cuts targeting low performers. Read the memo
Raw response: {
   "layoffs": "3,600",
   "reason": "Cutting low-performing staff members"
}
Extracted: layoffs=3,600, reason=Cutting low-performing staff members
Article 5/10: Meta to cut 10,000 jobs in

In [ ]:
final_results

{'meta': [{'ticker': 'meta',
   'title': 'A comprehensive list of 2025 tech layoffs | TechCrunch',
   'url': 'https://techcrunch.com/2025/05/21/tech-layoffs-2025-list/',
   'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from companies such as Playtika, Microsoft, Chegg, and Meta, resulting in a total of 16,084 job losses in February alone, with the year's total layoffs surpassing 150,000 across 549 companies, according to independent tracker Layoffs.fyi.",
   'date': '2025-05-21',
   'layoffs': '150,000',
   'reason': 'significant wave of layoffs in 2025',
   'liquidity': np.float64(-17.8),
   'sentiment': -0.9845200777053833},
  {'ticker': 'meta',
   'title': 'Meta cuts jobs at its Reality Labs division | Reuters',
   'url': 'https://www.reuters.com/business/world-at-work/meta-cuts-jobs-its-reality-labs-division-2025-04-24/',
   'summa

In [ ]:
for i in final_results.keys():
  for article in final_results[i]:
    article['liquidity']=calculate_liquidity_change(article)
    print(article['liquidity'])
    article['sentiment']=sentiment_analysis(article['summary'])



Article: A comprehensive list of 2025 tech layoffs | TechCrunch
Original: 2025-05-21 | Nearest trading day: 2025-05-21
Liquidity change: -17.8%
-17.8

Article: Meta cuts jobs at its Reality Labs division | Reuters
Original: 2025-04-24 | Nearest trading day: 2025-04-24
Liquidity change: -15.9%
-15.9

Article: Meta prepares for layoffs on Monday | Reuters
Original: 2025-02-07 | Nearest trading day: 2025-02-07
Liquidity change: -20.1%
-20.1

Article: Meta announces 5% cuts targeting low performers. Read the memo
Original: 2025-01-14 | Nearest trading day: 2025-01-14
Liquidity change: -10.1%
-10.1

Article: Meta to cut 10,000 jobs in second round of layoffs | Reuters
Original: 2023-03-14 | Nearest trading day: 2023-03-14
Liquidity change: 41.7%
41.7

Article: Meta layoffs: 10,000 more workers to be cut in restructuring
Original: 2023-03-14 | Nearest trading day: 2023-03-14
Liquidity change: 41.7%
41.7

Article: Google, Meta, Amazon, others have cut more than 104,000 employees
Original: 20

TypeError: 'ObjectId' object is not iterable

In [ ]:
final_results

{'meta': [{'ticker': 'meta',
   'title': 'A comprehensive list of 2025 tech layoffs | TechCrunch',
   'url': 'https://techcrunch.com/2025/05/21/tech-layoffs-2025-list/',
   'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from companies such as Playtika, Microsoft, Chegg, and Meta, resulting in a total of 16,084 job losses in February alone, with the year's total layoffs surpassing 150,000 across 549 companies, according to independent tracker Layoffs.fyi.",
   'date': '2025-05-21',
   'layoffs': '150,000',
   'reason': 'significant wave of layoffs in 2025',
   'liquidity': np.float64(-17.8),
   'sentiment': -0.9845200777053833},
  {'ticker': 'meta',
   'title': 'Meta cuts jobs at its Reality Labs division | Reuters',
   'url': 'https://www.reuters.com/business/world-at-work/meta-cuts-jobs-its-reality-labs-division-2025-04-24/',
   'summa

In [ ]:
layoffs_data = []
for ticker, articles in final_results.items():
    if ticker=='_id':
      continue
    processed_articles = []
    for article in articles:
        # Create a new dictionary without the '_id' field
        processed_article = {k: v for k, v in article.items() if k != '_id'}
        processed_articles.append(processed_article)
    layoffs_data.append({ticker: processed_articles})

In [ ]:
layoffs_data

[{'meta': [{'ticker': 'meta',
    'title': 'A comprehensive list of 2025 tech layoffs | TechCrunch',
    'url': 'https://techcrunch.com/2025/05/21/tech-layoffs-2025-list/',
    'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from companies such as Playtika, Microsoft, Chegg, and Meta, resulting in a total of 16,084 job losses in February alone, with the year's total layoffs surpassing 150,000 across 549 companies, according to independent tracker Layoffs.fyi.",
    'date': '2025-05-21',
    'layoffs': '150,000',
    'reason': 'significant wave of layoffs in 2025',
    'liquidity': np.float64(-17.8),
    'sentiment': -0.9845200777053833},
   {'ticker': 'meta',
    'title': 'Meta cuts jobs at its Reality Labs division | Reuters',
    'url': 'https://www.reuters.com/business/world-at-work/meta-cuts-jobs-its-reality-labs-division-2025-04-24/

#MonthlyLiqData


In [ ]:
tickers=['meta','google','microsoft','apple','amazon']

In [ ]:
company_to_ticker = {
    'meta': 'META',
    'amazon': 'AMZN',
    'microsoft': 'MSFT',
    'google': 'GOOGL',
    'apple': 'AAPL'
}

In [ ]:
import yfinance as yf
import pandas as pd

def get_monthly_liquidity(ticker, period="2y"):
    # Fetch daily historical data
    ticker = company_to_ticker[ticker.lower()]
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period, interval="1d")

    # Ensure the index is a datetime type
    hist.index = pd.to_datetime(hist.index)

    # Calculate daily spread
    hist['Spread'] = hist['High'] - hist['Low']

    # Group by year and month, then calculate average volume and average spread
    monthly_liquidity = hist.groupby([hist.index.year, hist.index.month]).agg({
        'Volume': 'mean',
        'Spread': 'mean',
    })
    monthly_liquidity['date'] = monthly_liquidity.index.map(
        lambda x: f"{x[0]}-{str(x[1]).zfill(2)}"  # e.g., "2024-06"
    )
    monthly_liquidity.index.set_names(['Year', 'Month'], inplace=True)
    monthly_liquidity.rename(columns={'Volume': 'Avg_Volume', 'Spread': 'Avg_Spread'}, inplace=True)

    return monthly_liquidity

In [ ]:
liquidity={}
for ticker in tickers:
  liquidity[ticker]=get_monthly_liquidity(ticker, period="3y")

In [ ]:
converted_data = {}
for key, value in liquidity.items():
  if isinstance(value, pd.DataFrame):
      converted_data[key] = value.to_dict(orient="records")
  else:
      converted_data[key] = value

In [ ]:
converted_data

{'meta': [{'Avg_Volume': 36963641.666666664,
   'Avg_Spread': 6.951403071879359,
   'date': '2022-06'},
  {'Avg_Volume': 29324120.0,
   'Avg_Spread': 6.160465369046006,
   'date': '2022-07'},
  {'Avg_Volume': 23996647.826086957,
   'Avg_Spread': 5.4369853908219605,
   'date': '2022-08'},
  {'Avg_Volume': 30238800.0,
   'Avg_Spread': 5.384621519807248,
   'date': '2022-09'},
  {'Avg_Volume': 52745819.047619045,
   'Avg_Spread': 5.486048870606757,
   'date': '2022-10'},
  {'Avg_Volume': 50656723.809523806,
   'Avg_Spread': 3.9627518171633396,
   'date': '2022-11'},
  {'Avg_Volume': 31096180.95238095,
   'Avg_Spread': 3.9314708242333247,
   'date': '2022-12'},
  {'Avg_Volume': 27438530.0,
   'Avg_Spread': 4.141978969653569,
   'date': '2023-01'},
  {'Avg_Volume': 40640573.684210524,
   'Avg_Spread': 6.415551396443679,
   'date': '2023-02'},
  {'Avg_Volume': 30002304.347826086,
   'Avg_Spread': 5.967527587234378,
   'date': '2023-03'},
  {'Avg_Volume': 23509889.47368421,
   'Avg_Spread': 4

In [ ]:
liquidity_data=[]
for i in converted_data.keys():
  if i=='_id':
    continue
  for key, articles in converted_data[i]:
    processed_articles = []
    for article in articles:
      processed_article = {k: v for k, v in article.items() if k != '_id'}
      processed_articles.append(processed_article)
  liquidity_data.append({i:processed_articles})

In [ ]:
liquidity_data

[{'meta': [{'Avg_Volume': 36963641.666666664,
    'Avg_Spread': 6.951403071879359,
    'date': '2022-06'},
   {'Avg_Volume': 29324120.0,
    'Avg_Spread': 6.160465369046006,
    'date': '2022-07'},
   {'Avg_Volume': 23996647.826086957,
    'Avg_Spread': 5.4369853908219605,
    'date': '2022-08'},
   {'Avg_Volume': 30238800.0,
    'Avg_Spread': 5.384621519807248,
    'date': '2022-09'},
   {'Avg_Volume': 52745819.047619045,
    'Avg_Spread': 5.486048870606757,
    'date': '2022-10'},
   {'Avg_Volume': 50656723.809523806,
    'Avg_Spread': 3.9627518171633396,
    'date': '2022-11'},
   {'Avg_Volume': 31096180.95238095,
    'Avg_Spread': 3.9314708242333247,
    'date': '2022-12'},
   {'Avg_Volume': 27438530.0,
    'Avg_Spread': 4.141978969653569,
    'date': '2023-01'},
   {'Avg_Volume': 40640573.684210524,
    'Avg_Spread': 6.415551396443679,
    'date': '2023-02'},
   {'Avg_Volume': 30002304.347826086,
    'Avg_Spread': 5.967527587234378,
    'date': '2023-03'},
   {'Avg_Volume': 235098

#Insights


In [ ]:
tickers=['meta','google','microsoft','apple','amazon']

In [ ]:
from datetime import datetime

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return datetime.min

In [ ]:
insightful_results={}
seen_links = set()
for ticker in tickers:
  for i in range(7):
    try:
       print(ticker)
       google_results = fetch_from_google_cse(f"broad news from {ticker} -stock -price -market",start=i*10+1)
    except Exception as e:
      print(e)
    ticker_results=[]
    for item in google_results:
        link = item['link']
        print(item['title'])
        if link not in seen_links:
            date = None
            try:
                metatags = item.get('pagemap', {}).get('metatags', [{}])
                date = metatags[0].get('article:published_time') if metatags else None
            except (KeyError, IndexError):
                pass

            ticker_results.append({
                "ticker": ticker,
                "title": item['title'],
                "url": link,
                "summary": summarize(link),
                "date": date[:10] if date else None # Added date field
            })
            seen_links.add(link)
  sorted_results=sorted(ticker_results,key=lambda x: parse_date(x.get('date')),reverse=True)
  insightful_results[ticker]=sorted_results

meta
Facebook renamed its 'news feed' to just 'feed' | TechCrunch
Meta launches program to encourage startups to use its Llama AI ...
Meta adds Stripe CEO Patrick Collison and Dina Powell McCormick ...
Meta tests facial recognition for spotting 'celeb-bait' ads scams and ...
Mark Zuckerberg defends Meta's latest pivot in three-hour Joe ...
Meta starts blocking the news in Canada — its latest standoff with ...
Apple might partner with Meta on AI | TechCrunch
Scale AI confirms 'significant' investment from Meta, says CEO ...
Meta's V-JEPA 2 model teaches AI to understand its surroundings ...
Meta fined $263M over 2018 security breach that affected ~3M EU ...
meta
Court filings show Meta paused efforts to license books for AI ...
Meta eases up on issuing 'first strikes' for Facebook users and ...
Here's how to watch LlamaCon, Meta's first AI developer event ...
Meta won't say whether it trains AI on smart glasses photos ...
Meta's benchmarks for its new AI models are a bit misleading ...


In [ ]:
insightful_results

{'meta': [{'ticker': 'meta',
   'title': 'In AI copyright case, Zuckerberg turns to YouTube for his defense ...',
   'url': 'https://techcrunch.com/2025/01/15/in-ai-copyright-case-zuckerberg-turns-to-youtube-for-his-defense/',
   'summary': "Meta CEO Mark Zuckerberg has defended his company's use of a copyrighted e-book dataset, LibGen, by citing YouTube's efforts to remove pirated content, stating that the vast majority of content on the platform is licensed and legitimate, in a deposition related to the AI copyright case Kadrey v. Meta Platforms.\n\nKey entities:\n\n* Mark Zuckerberg, Meta CEO\n* YouTube\n* Meta Platforms\n* LibGen, a copyrighted e-book dataset\n* Llama, Meta's AI models\n\nFinancial impacts and numbers:\n\n* None mentioned in the article.",
   'date': '2025-01-16'},
  {'ticker': 'meta',
   'title': 'Stainless helps build SDKs for OpenAI, Anthropic, and Meta ...',
   'url': 'https://techcrunch.com/2024/12/10/stainless-helps-build-sdks-for-openai-anthropic-and-meta/',

In [ ]:
train_layoffs=['''it resulted in thousands of federal job cuts, winds down, mass layoffs are still roiling corporate America.Though many companies declined to provide specific
reasoning for announced workforce reductions — instead lumping the layoffs in with larger cost-cutting strategies or growth plans — tech leaders are starting to cite artificial
intelligence as a clear consideration in hiring and head-count reductions.Pampers and Tide maker Procter & Gamble Thursday it will cut 7,000 jobs, or about 15% of its
non-manufacturing workforce, over the next two years as part of a restructuring program.Microsoft on May 13 that it would reduce its workforce by about 6,000 staffers, totaling
about 3% of employees across all teams, levels and geographies.Citi group in a statement Thursday it plans to reduce its staff by around 3,500 positions in China.On May 21,
Reuters reported that Walmart was planning to slash about 1,500 jobs in an effort to simplify operations.Siemiatkowski in May that the 40% cut in head count is due not only
to AI but also to attrition, after the company instituted a hiring freeze.Cybersecurity software maker CrowdStrike on May 7 announced plans to cut 500 employees, or about 5% of
its staff.Online education firm Chegg on May 12 that it would lay off 248 employees, or about 22% of its workforce. The cuts come as AI-powered tools such as OpenAI’s ChatGPT
take over education.Amazon has confirmed a fresh wave of layoffs, this time affecting staff in its Books division, including the Kindle and Goodreads teams. The cuts, which
reportedly affect fewer than 100 employees.The layoffs are part of a broader restructuring strategy, which has seen targeted reductions across different segments rather than sweeping
company-wide cuts. According to a report from Layoffs.fyi, more than 61,220 tech workers have lost their jobs in 2025 so far.Canada’s Hudson’s Bay plans to lay off 8,347
employees, or 89% of its workforce.Paramount Global is cutting its U.S.-based staff by 3.5%, or several hundred employees, in the latest round of layoffs at the media.In
August, Paramount began the process of reducing its U.S.-based workforce by 15%.Nokia’s senior vice president of human resources at the time, flew in to talk about the layoff with
the plant’s 2,300 employees.    ''']

In [ ]:
train_legal = [
    '''Meta will face off against the U.S. Federal Trade Commission on Monday in a high-stakes antitrust trial that could result in the company divesting Instagram and WhatsApp.
     The trial will test the boundaries of the U.S.’s antitrust laws pertaining to corporate acquisitions, said Prasad Krishnamurthy, a law professor at U.C. O’Carroll’s lawsuit
      alleged the technology conglomerate was ignoring her legal right to object to the processing and continued use of her personal data. In the wake of the settlement which was
       agreed just days before the claim was due to be heard in the British High Court. Took the stand on Monday at a high-stakes trial in Washington over U.S. antitrust enforcers.''',
    '''Google is again facing legal troubles and this time over its stronghold over the online advertising ecosystem. Is currently facing a lawsuit over falsely advertising the
     Apple Intelligence features. In a 115-page ruling issued on Thursday, Judge Leonie Brinkema of the US District Court for the Eastern District of Virginia concluded that had
     engaged in illegal conduct to cement its control over key advertising technologies. The US federal judge agreed with the government in two of the three areas under scrutiny. This case is one of several antitrust actions facing. In a separate case last August, another federal judge'''
]

In [ ]:
train_data={"layoffs":train_layoffs,"legal issues":train_legal}

In [ ]:
critical_event_keywords = {
    'layoffs': ['job cuts', 'restructuring'],
    'legal issues': [ 'legal action','sue', 'settlement', 'court'],
}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def suggest_keywords(train_text, top_n=3):
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2),min_df=1,max_df=1,sublinear_tf=True)
    tfidf_matrix = vectorizer.fit_transform(train_text)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_sum = np.asarray(tfidf_matrix.sum(axis=0)).flatten()
    top_indices = tfidf_sum.argsort()[::-1][:top_n]
    suggested_keywords = [feature_names[i] for i in top_indices]
    return suggested_keywords


In [ ]:
def process_label(text):
  labels=[]
  for i in critical_event_keywords.keys():
    for j in critical_event_keywords[i]:
       if j in text :
         labels.append(i)
  return labels

In [ ]:
for i in critical_event_keywords.keys():
    for j in train_data[i]:
       for key in suggest_keywords([j]): # Pass j as a list
         if key not in critical_event_keywords[i]:
           critical_event_keywords[i].append(key)

In [ ]:
critical_event_keywords

{'layoffs': ['job cuts', 'restructuring', 'employees', 'layoffs', 'workforce'],
 'legal issues': ['legal action',
  'sue',
  'settlement',
  'court',
  'trial',
  'antitrust',
  'high',
  'judge',
  'advertising',
  'facing']}

In [ ]:
for i in insightful_results.keys():
  for article in insightful_results[i]:
    article['labels']=process_label(article['summary'])
    article['sentiment']=sentiment_analysis(article['summary'])

In [ ]:
for i in insightful_results.keys():
  for article in insightful_results[i]:
    x = calculate_liquidity_change(article)
    print(x)
    if x is not None and x.get('liquidity') is not None:
      x_liquidity_pct_change = x['liquidity']
      print(x_liquidity_pct_change)
      article['liquidity'] = x_liquidity_pct_change
    else:
      article['liquidity'] = None


Article: In AI copyright case, Zuckerberg turns to YouTube for his defense ...
Original: 2025-01-16 | Nearest trading day: 2025-01-16
Liquidity change: 7.0%
{'ticker': 'meta', 'title': 'In AI copyright case, Zuckerberg turns to YouTube for his defense ...', 'url': 'https://techcrunch.com/2025/01/15/in-ai-copyright-case-zuckerberg-turns-to-youtube-for-his-defense/', 'summary': "Meta CEO Mark Zuckerberg has defended his company's use of a copyrighted e-book dataset, LibGen, by citing YouTube's efforts to remove pirated content, stating that the vast majority of content on the platform is licensed and legitimate, in a deposition related to the AI copyright case Kadrey v. Meta Platforms.\n\nKey entities:\n\n* Mark Zuckerberg, Meta CEO\n* YouTube\n* Meta Platforms\n* LibGen, a copyrighted e-book dataset\n* Llama, Meta's AI models\n\nFinancial impacts and numbers:\n\n* None mentioned in the article.", 'date': '2025-01-16', 'labels': [], 'sentiment': -0.994657039642334, 'liquidity': np.float

In [ ]:
insights_data=[]
for i in insightful_results.keys():
  if
  insights_data.append({i:insightful_results[i]})

In [ ]:
insights_data = []
for ticker, articles in insightful_results.items():
    if ticker=='_id':
      continue
    processed_articles = []
    for article in articles:
        # Create a new dictionary without the '_id' field
        processed_article = {k: v for k, v in article.items() if k != '_id'}
        processed_articles.append(processed_article)
    insights_data.append({ticker: processed_articles})

[{'meta': [{'ticker': 'meta',
    'title': 'In AI copyright case, Zuckerberg turns to YouTube for his defense ...',
    'url': 'https://techcrunch.com/2025/01/15/in-ai-copyright-case-zuckerberg-turns-to-youtube-for-his-defense/',
    'summary': "Meta CEO Mark Zuckerberg has defended his company's use of a copyrighted e-book dataset, LibGen, by citing YouTube's efforts to remove pirated content, stating that the vast majority of content on the platform is licensed and legitimate, in a deposition related to the AI copyright case Kadrey v. Meta Platforms.\n\nKey entities:\n\n* Mark Zuckerberg, Meta CEO\n* YouTube\n* Meta Platforms\n* LibGen, a copyrighted e-book dataset\n* Llama, Meta's AI models\n\nFinancial impacts and numbers:\n\n* None mentioned in the article.",
    'date': '2025-01-16',
    'labels': [],
    'sentiment': -0.994657039642334,
    'liquidity': np.float64(7.0)},
   {'ticker': 'meta',
    'title': 'Stainless helps build SDKs for OpenAI, Anthropic, and Meta ...',
    'url

#Uploading On MongoDB

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
#There is readymade data below, run the below section if running entire codebase is as issue

In [ ]:
uri = "Upload on a mongoDB server"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
# Step 1: Connect to MongoDB
db = client["Scrapping"] # Changed to correct casing
topnews_db = db["topnews"]
insights_db = db["insights"]
stockdata_db = db["stockdata"]
layoffs_db = db["layoffs"]

# Step 2: Insert Data
layoffs_db.insert_many(layoffs_data)
insights_db.insert_many(insights_data)
topnews_db.insert_many(topnews)
stockdata_db.insert_many(liquidity_data)
print("Data successfully uploaded to MongoDB!")

Data successfully uploaded to MongoDB!


#StoringGeneratedData(safety)

In [ ]:
topnews=[{'ticker': 'Microsoft',
  'title': 'Apple, Meta fined as EU presses ahead with tech probes ',
  'url': 'https://www.reuters.com/sustainability/boards-policy-regulation/apple-fined-570-million-meta-228-million-breaching-eu-law-2025-04-23/',
  'summary': "Here is a concise news summary:\n\nThe UK's competition watchdog has launched a probe into the $13.25 billion merger between advertising giants Omnicom and Interpublic, to assess whether the deal could lead to a reduction in competition in the UK market, as Apple and Meta face fines for breaching EU antitrust rules.",
  'date': '2025-04-23',
  'sentiment': 0.833164393901825,
  'liquidity': np.float64(2.9)},
 {'ticker': 'Meta',
  'title': 'The biggest questions facing Big Tech ahead of earnings',
  'url': 'https://www.cnbc.com/2025/04/21/tech-earnings-trump-tariffs-tesla-apple-alphabet-meta-microsoft-amazon-nvidia.html',
  'summary': "As tech's megacap companies enter first-quarter earnings season, uncertainty reigns supreme due to President Trump's on-again, off-again approach to tariffs, which has created market chaos, with key entities such as Tesla, Alphabet, Meta, Microsoft, Amazon, Apple, and Nvidia facing financial impacts and uncertainty in their revenue and earnings forecasts.",
  'date': '2025-04-21',
  'sentiment': -0.7967345714569092,
  'liquidity': np.float64(9.3)},
 {'ticker': 'Amazon',
  'title': 'Tech megacaps to spend more than $300 billion in 2025 to win in AI',
  'url': 'https://www.cnbc.com/2025/02/08/tech-megacaps-to-spend-more-than-300-billion-in-2025-to-win-in-ai.html',
  'summary': 'Tech giants Meta, Amazon, Alphabet, and Microsoft plan to spend a combined $320 billion on artificial intelligence technologies and datacenter buildouts in 2025, a significant increase from $230 billion in 2024, as they race to expand their AI capabilities and stay ahead in the competitive market.',
  'date': '2025-02-08',
  'sentiment': 0.9902418255805969,
  'liquidity': np.float64(-15.6)},
 {'ticker': 'CNBC',
  'title': "Cramer's week ahead: Earnings from Alphabet, Meta, Apple ...",
  'url': 'https://www.cnbc.com/2024/10/25/cramers-week-ahead-earnings-from-alphabet-meta-apple-microsoft-and-amazon.html',
  'summary': "CNBC's Jim Cramer advises investors to remain cautious ahead of a massive earnings week, with major tech companies Alphabet, Meta, Apple, Microsoft, and Amazon set to report, while also highlighting the importance of the nonfarm payroll report and warning of potential pitfalls in the earnings season.",
  'date': '2024-10-25',
  'sentiment': 0.987820565700531,
  'liquidity': None},
 {'ticker': 'Google',
  'title': 'Big Tech farms out AI power build, keeps the risk ',
  'url': 'https://www.reuters.com/breakingviews/big-tech-farms-out-ai-power-build-keeps-risk-2024-10-03/',
  'summary': "The Reserve Bank of India, led by Governor Sanjay Malhotra, surprised markets by slashing its key policy rate by 50 basis points, prioritizing GDP growth over inflation targeting, a move that may have significant financial implications for the country's economy.",
  'date': '2024-10-03',
  'sentiment': 0.915814220905304,
  'liquidity': np.float64(-1.1)},
 {'ticker': 'TechCrunch Daily News',
  'title': "Early sign-ups to EU's AI Pact include Amazon, Google, Microsoft ...",
  'url': 'https://techcrunch.com/2024/09/25/early-sign-ups-to-eus-ai-pact-include-amazon-google-microsoft-and-openai-but-apple-and-meta-are-missing/',
  'summary': "The European Commission has announced the first 100-plus signatories to the AI Pact, including tech giants Amazon, Google, Microsoft, and OpenAI, which aim to voluntarily commit to implementing the bloc's AI rulebook and best practices, with Apple and Meta notably absent from the list.",
  'date': '2024-09-25',
  'sentiment': -0.9967027306556702,
  'liquidity': None},
 {'ticker': 'Apple',
  'title': "AI engineers face burnout in 'rat race' to stay competitive hits tech",
  'url': 'https://www.cnbc.com/2024/05/03/ai-engineers-face-burnout-as-rat-race-to-stay-competitive-hits-tech.html',
  'summary': 'Here is a concise news summary:\n\nAI engineers at top tech companies, including Amazon, Google, and Microsoft, are reporting burnout and rushed rollouts as they struggle to keep up with the intense pressure to stay competitive in the rapidly evolving AI landscape, with many feeling undervalued and overworked as they prioritize speed over quality and ignore potential risks and harms.',
  'date': '2024-05-03',
  'sentiment': -0.9649980068206787,
  'liquidity': np.float64(-11.9)},
 {'ticker': 'Google',
  'title': "Inside Big Tech's underground race to buy AI training data ",
  'url': 'https://www.reuters.com/technology/inside-big-techs-underground-race-buy-ai-training-data-2024-04-05/',
  'summary': 'The US is pushing Vietnam to reduce its reliance on Chinese technology in devices assembled in the country for export to America, as part of tariff talks, with the goal of promoting the use of "ethically sourced" AI training data, a move that has sparked concerns among tech companies and experts.',
  'date': '2024-04-05',
  'sentiment': 0.978543221950531,
  'liquidity': np.float64(-12.4)},
 {'ticker': 'Microsoft',
  'title': 'Google, Meta, Amazon and other tech companies have laid off more ...',
  'url': 'https://www.cnbc.com/2023/01/18/tech-layoffs-microsoft-amazon-meta-others-have-cut-more-than-60000.html',
  'summary': 'Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nTech giants Google, Meta, Amazon, and others have collectively laid off over 104,000 employees in the past year, with Amazon cutting 27,000 jobs, Meta eliminating 21,000, Alphabet cutting 12,000, and Microsoft letting go of 10,000, among others, as the industry adapts to a new reality of slowing growth, higher interest rates, and recession fears.',
  'date': '2023-01-18',
  'sentiment': 0.930809736251831,
  'liquidity': np.float64(17.9)},
 {'ticker': 'Amazon',
  'title': 'Google, Facebook, Microsoft top EU lobbying spending - study ...',
  'url': 'https://www.reuters.com/technology/google-facebook-microsoft-top-eu-lobbying-spending-study-2021-08-30/',
  'summary': 'Here is a concise news summary:\n\nGoogle, Facebook, and Microsoft topped the list of companies that spent the most on lobbying in the European Union, with combined expenditures of over €100 million, according to a study, as the tech giants seek to influence EU policies and regulations.',
  'date': '2021-08-31',
  'sentiment': 0.9466316103935242,
  'liquidity': np.float64(6.5)}]

NameError: name 'np' is not defined

In [ ]:
layoffs_data=[{'meta': [{'ticker': 'meta',
    'title': 'A comprehensive list of 2025 tech layoffs | TechCrunch',
    'url': 'https://techcrunch.com/2025/05/21/tech-layoffs-2025-list/',
    'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from companies such as Playtika, Microsoft, Chegg, and Meta, resulting in a total of 16,084 job losses in February alone, with the year's total layoffs surpassing 150,000 across 549 companies, according to independent tracker Layoffs.fyi.",
    'date': '2025-05-21',
    'layoffs': '150,000',
    'reason': 'significant wave of layoffs in 2025',
    'liquidity': np.float64(-17.8),
    'sentiment': -0.9845200777053833},
   {'ticker': 'meta',
    'title': 'Meta cuts jobs at its Reality Labs division | Reuters',
    'url': 'https://www.reuters.com/business/world-at-work/meta-cuts-jobs-its-reality-labs-division-2025-04-24/',
    'summary': "Here is a concise news summary:\n\nMeta, the parent company of Facebook and Instagram, has cut jobs at its Reality Labs division, a unit focused on developing virtual and augmented reality technologies, with no official word on the number of layoffs, but sources suggest it's a significant reduction.",
    'date': '2025-04-24',
    'layoffs': '10,000',
    'reason': 'significant reduction in workforce',
    'liquidity': np.float64(-15.9),
    'sentiment': -0.9839142560958862},
   {'ticker': 'meta',
    'title': 'Meta prepares for layoffs on Monday | Reuters',
    'url': 'https://www.reuters.com/technology/meta-prepares-layoffs-monday-internal-memo-2025-02-07/',
    'summary': 'Here is a concise news summary:\n\nMeta is preparing to lay off employees on Monday, while the US is pushing Vietnam to reduce its reliance on Chinese technology in devices assembled in the country for export to America, according to sources, with no specific financial numbers disclosed.',
    'date': '2025-02-07',
    'layoffs': '10,000',
    'reason': 'no specific financial numbers disclosed',
    'liquidity': np.float64(-20.1),
    'sentiment': -0.9878451228141785},
   {'ticker': 'meta',
    'title': 'Meta announces 5% cuts targeting low performers. Read the memo',
    'url': 'https://www.cnbc.com/2025/01/14/meta-targeting-lowest-performing-employees-in-latest-round-of-layoffs.html',
    'summary': 'Meta, the parent company of Facebook and Instagram, is set to cut around 5% of its workforce, or approximately 3,600 employees, in preparation for what CEO Mark Zuckerberg has described as an "intense year", with the layoffs focusing on the company\'s lowest-performing staff members.',
    'date': '2025-01-14',
    'layoffs': '3,600',
    'reason': 'Cutting low-performing staff members',
    'liquidity': np.float64(-10.1),
    'sentiment': -0.9987290501594543},
   {'ticker': 'meta',
    'title': 'Meta to cut 10,000 jobs in second round of layoffs | Reuters',
    'url': 'https://www.reuters.com/technology/facebook-parent-meta-lay-off-10000-employees-second-round-job-cuts-2023-03-14/',
    'summary': 'Meta, the parent company of Facebook and Instagram, is cutting 10,000 jobs, its second round of layoffs this year, in a move that is expected to satisfy investors and help the company reduce costs amid a challenging economic environment.',
    'date': '2023-03-14',
    'layoffs': '10,000',
    'reason': 'to satisfy investors and help the company reduce costs amid a challenging economic environment',
    'liquidity': np.float64(41.7),
    'sentiment': 0.987464189529419},
   {'ticker': 'meta',
    'title': 'Meta layoffs: 10,000 more workers to be cut in restructuring',
    'url': 'https://www.cnbc.com/2023/03/14/meta-layoffs-10000-more-workers-to-be-cut-in-restructuring.html',
    'summary': 'Meta, the parent company of Facebook, announced plans to lay off an additional 10,000 workers, following a previous round of cuts in November, and expects to incur restructuring costs of $3 billion to $5 billion, as CEO Mark Zuckerberg warns of potential economic instability for "many years".',
    'date': '2023-03-14',
    'layoffs': '10,000',
    'reason': 'Restructuring due to potential economic instability',
    'liquidity': np.float64(41.7),
    'sentiment': -0.9987033605575562},
   {'ticker': 'meta',
    'title': 'Google, Meta, Amazon, others have cut more than 104,000 employees',
    'url': 'https://www.cnbc.com/2023/01/18/tech-layoffs-microsoft-amazon-meta-others-have-cut-more-than-60000.html',
    'summary': 'Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nTech giants Google, Meta, Amazon, and others have collectively laid off over 104,000 employees in the past year, with Amazon cutting 27,000 jobs, Meta cutting 21,000, Alphabet cutting 12,000, and Microsoft cutting 10,000, among others, as the industry adapts to a new reality of slowing growth, higher interest rates, and recession fears.',
    'date': '2023-01-18',
    'layoffs': '27,000',
    'reason': 'slowing growth, higher interest rates, and recession fears',
    'liquidity': np.float64(28.8),
    'sentiment': 0.9159961938858032},
   {'ticker': 'meta',
    'title': 'FACTBOX Tech firms, Wall Street lead job cuts in Corporate America ...',
    'url': 'https://www.reuters.com/world/us/corporate-america-lays-off-thousands-recession-worries-mount-2023-01-04/',
    'summary': "Tech firms and Wall Street led the charge in Corporate America's job cuts, with major companies such as Google, Microsoft, and Goldman Sachs announcing thousands of layoffs, while Indian tribes in Nagaland held talks with a British museum to secure the return of ancestral remains taken during the colonial era.",
    'date': '2023-01-10',
    'layoffs': '10,000',
    'reason': "Tech firms and Wall Street led the charge in Corporate America's job cuts",
    'liquidity': np.float64(-0.5),
    'sentiment': -0.9828416109085083},
   {'ticker': 'meta',
    'title': 'Meta cuts 11,000 jobs as it sinks more money into the metaverse ...',
    'url': 'https://www.reuters.com/technology/meta-cut-more-than-11000-jobs-one-biggest-us-layoffs-this-year-2022-11-09/',
    'summary': 'Meta, the parent company of Facebook and Instagram, has announced a major restructuring, cutting 11,000 jobs, or about 13% of its workforce, as it shifts focus to developing its metaverse technology, a move that will result in a significant cash burn of around $10 billion in the current quarter.',
    'date': '2022-11-10',
    'layoffs': '11,000',
    'reason': 'Shifting focus to developing metaverse technology',
    'liquidity': np.float64(6.5),
    'sentiment': -0.9969149827957153},
   {'ticker': 'meta',
    'title': "Meta laying off more than 11,000 employees: Read Zuckerberg's letter",
    'url': 'https://www.cnbc.com/2022/11/09/meta-to-lay-off-more-than-11000-thousand-employees.html',
    'summary': 'Meta, the parent company of Facebook, is laying off over 11,000 employees, or 13% of its workforce, as part of a cost-cutting effort, with CEO Mark Zuckerberg citing the need to become a "leaner and more efficient company" amidst rising costs and expenses.',
    'date': '2022-11-09',
    'layoffs': '11,000',
    'reason': 'becoming a leaner and more efficient company',
    'liquidity': np.float64(48.8),
    'sentiment': -0.9924408197402954}]},
 {'google': [{'ticker': 'google',
    'title': 'Google cuts about 200 staff in global business unit, The Information ...',
    'url': 'https://www.reuters.com/technology/google-cuts-about-200-staff-global-business-unit-information-reports-2025-05-07/',
    'summary': 'Here is a concise news summary:\n\nGoogle has cut around 200 staff from its global business unit, according to a report by The Information, while the US is pushing Vietnam to reduce its reliance on Chinese technology in devices assembled in the country for export to America, as part of tariff talks.',
    'date': '2025-05-07',
    'layoffs': '200',
    'reason': 'restructuring',
    'liquidity': np.float64(44.6),
    'sentiment': 0.7060288190841675},
   {'ticker': 'google',
    'title': 'Google makes cuts to HR and cloud units',
    'url': 'https://www.cnbc.com/2025/02/27/google-makes-cuts-to-hr-and-cloud-units.html',
    'summary': 'Google has announced layoffs in its HR and cloud units as part of ongoing cost-cutting measures, offering voluntary exit programs to U.S.-based employees in People Operations and making cuts to several teams within its cloud unit, affecting operations support staff and potentially relocating some roles to India and Mexico City.',
    'date': '2025-02-27',
    'layoffs': 'not mentioned',
    'reason': 'part of ongoing cost-cutting measures',
    'liquidity': np.float64(36.1),
    'sentiment': -0.9988440275192261},
   {'ticker': 'google',
    'title': "Google employees petition for 'job security' ahead of expected cuts",
    'url': 'https://www.cnbc.com/2025/01/28/google-employees-petition-for-job-security-ahead-of-expected-cuts-.html',
    'summary': 'Here is a concise news summary:\n\nGoogle employees have launched an internal petition, signed by over 1,250 workers, calling for job security measures ahead of expected cost cuts, including buyouts, guaranteed severance, and no forced performance reviews, amid concerns over ongoing layoffs and uncertainty at the company.',
    'date': '2025-01-28',
    'layoffs': '0',
    'reason': 'no layoffs mentioned',
    'liquidity': np.float64(-21.6),
    'sentiment': 0.9663835167884827},
   {'ticker': 'google',
    'title': 'Google cuts at least 100 jobs across cloud unit, sources say',
    'url': 'https://www.cnbc.com/2024/06/03/google-cuts-at-least-100-jobs-across-cloud-unit-sources-say.html',
    'summary': 'Google has cut at least 100 jobs across its fast-growing cloud unit, a move that comes as the company records record profits and faces pressure to continue accelerating growth in the competitive artificial intelligence market.',
    'date': '2024-06-03',
    'layoffs': '100',
    'reason': 'pressure to continue accelerating growth in the competitive artificial intelligence market',
    'liquidity': np.float64(-23.9),
    'sentiment': 0.5926238894462585},
   {'ticker': 'google',
    'title': "Google cuts hundreds of 'Core' workers, moves jobs to India, Mexico",
    'url': 'https://www.cnbc.com/2024/05/01/google-cuts-hundreds-of-core-workers-moves-jobs-to-india-mexico.html',
    'summary': 'Google has laid off at least 200 employees from its "Core" teams, responsible for building the technical foundation behind its flagship products, with some positions being moved to India and Mexico, as the company continues to downsize despite reporting its fastest growth rate since early 2022.',
    'date': '2024-05-01',
    'layoffs': '200',
    'reason': 'Downsizing despite reporting fastest growth rate since early 2022',
    'liquidity': np.float64(-38.8),
    'sentiment': -0.9991040825843811},
   {'ticker': 'google',
    'title': "Google CEO says job cuts needed in 2024 to serve 'ambitious goals'",
    'url': 'https://www.cnbc.com/2024/01/17/google-ceo-says-job-cuts-needed-in-2024-to-serve-ambitious-goals.html',
    'summary': 'Google CEO Sundar Pichai warned employees that the company will make further job cuts in 2024 to reach its "ambitious goals" and invest in areas like artificial intelligence, following a series of layoffs that began in January 2023 and eliminated around 12,000 jobs, or roughly 6% of the full-time workforce.',
    'date': '2024-01-18',
    'layoffs': '12,000',
    'reason': 'to reach ambitious goals and invest in areas like artificial intelligence',
    'liquidity': np.float64(13.4),
    'sentiment': -0.9989175796508789},
   {'ticker': 'google',
    'title': 'Google layoffs: Hundreds of jobs cut across engineering and hardware',
    'url': 'https://www.cnbc.com/2024/01/11/google-layoffs-hundreds-of-jobs-cut-across-engineering-and-hardware.html',
    'summary': 'Google has cut hundreds of jobs across its engineering, hardware, and Google Assistant teams, as part of a cost-cutting effort to prioritize its "biggest product priorities" and reduce headcount growth, with shares of Alphabet, its parent company, closing down less than 1% on Thursday.',
    'date': '2024-01-11',
    'layoffs': 'hundreds',
    'reason': "cost-cutting effort to prioritize 'biggest product priorities'",
    'liquidity': np.float64(-9.6),
    'sentiment': -0.9992412328720093},
   {'ticker': 'google',
    'title': 'Google workers in London stage walkout over job cuts | Reuters',
    'url': 'https://www.reuters.com/technology/google-workers-london-stage-walkout-over-job-cuts-2023-04-04/',
    'summary': "Here is a concise news summary:\n\nGoogle workers in London staged a walkout on Wednesday to protest the company's decision to cut jobs, with employees citing concerns over the impact on the company's culture and the lack of transparency in the layoffs.",
    'date': '2023-04-04',
    'layoffs': 'not applicable',
    'reason': 'lack of transparency in layoffs',
    'liquidity': np.float64(26.2),
    'sentiment': -0.8986610174179077},
   {'ticker': 'google',
    'title': 'Alphabet cuts 12,000 jobs after pandemic hiring spree, refocuses on ...',
    'url': 'https://www.reuters.com/business/google-parent-lay-off-12000-workers-memo-2023-01-20/',
    'summary': "Alphabet, the parent company of Google, has cut 12,000 jobs, or about 6.4% of its workforce, citing a need to refocus on artificial intelligence and other priorities, following a pandemic-era hiring spree that saw the company's workforce swell to over 190,000 employees.",
    'date': '2023-01-20',
    'layoffs': '12,000',
    'reason': 'refocus on artificial intelligence and other priorities',
    'liquidity': np.float64(8.5),
    'sentiment': -0.998867392539978}]},
 {'microsoft': [{'ticker': 'microsoft',
    'title': 'Corporate layoffs: Here are the companies making job cuts',
    'url': 'https://www.cnbc.com/2025/06/05/corporate-layoffs-companies-making-job-cuts.html',
    'summary': 'Here is a concise news summary:\n\nIn a bid to trim costs amid global economic uncertainty, several major companies, including Procter & Gamble, Microsoft, Citigroup, Walmart, Klarna, CrowdStrike, Disney, Chegg, Amazon, and Warner Bros. Discovery, have announced layoffs, with Procter & Gamble cutting 7,000 jobs, Microsoft reducing its workforce by 6,000 staffers, and Citigroup eliminating 3,500 positions in China, as companies seek to adapt to the changing economic landscape.',
    'date': '2025-06-05',
    'layoffs': '10,000',
    'reason': 'trim costs amid global economic uncertainty',
    'liquidity': np.float64(18.7),
    'sentiment': 0.8521004915237427},
   {'ticker': 'microsoft',
    'title': 'Microsoft laying off about 6,000 people, or 3% of its workforce',
    'url': 'https://www.cnbc.com/2025/05/13/microsoft-is-cutting-3percent-of-workers-across-the-software-company.html',
    'summary': 'Microsoft announced it is laying off approximately 6,000 employees, or 3% of its global workforce, as part of a restructuring effort aimed at reducing layers of management and positioning the company for success in a dynamic market, despite reporting better-than-expected quarterly results and an upbeat forecast.',
    'date': '2025-05-13',
    'layoffs': '6,000',
    'reason': 'reducing layers of management and positioning the company for success',
    'liquidity': np.float64(-8.4),
    'sentiment': -0.9768134355545044},
   {'ticker': 'microsoft',
    'title': 'Microsoft to lay off around 3% of workforce in organization-wide cuts ...',
    'url': 'https://www.reuters.com/business/world-at-work/microsoft-lay-off-3-workforce-cnbc-reports-2025-05-13/',
    'summary': "Microsoft is set to lay off around 3% of its workforce, while the UK's competition watchdog has launched a probe into the proposed $13.25 billion merger between advertising giants Omnicom and Interpublic, citing concerns over potential reductions in competition in the UK market.",
    'date': '2025-05-13',
    'layoffs': '10,000',
    'reason': 'organization-wide cuts',
    'liquidity': np.float64(-8.4),
    'sentiment': -0.9968928098678589},
   {'ticker': 'microsoft',
    'title': 'Microsoft pauses hiring in U.S. consulting unit, memo says',
    'url': 'https://www.cnbc.com/2025/01/14/microsoft-pauses-hiring-in-us-consulting-unit-memo-says.html',
    'summary': "Microsoft has paused hiring in its US consulting unit, a move aimed at reducing costs, as part of a broader cost-cutting plan, following a week of layoffs that affected less than 1% of its workforce, with the company's consulting division expected to cut marketing and non-billable external resource spend by 35% and hold off on hiring new employees and back-filling roles.",
    'date': '2025-01-14',
    'layoffs': 'less than 1%',
    'reason': 'reducing costs',
    'liquidity': np.float64(7.7),
    'sentiment': -0.9991655349731445},
   {'ticker': 'microsoft',
    'title': 'Microsoft confirms performance-based job cuts across departments',
    'url': 'https://www.cnbc.com/2025/01/08/microsoft-confirms-performance-based-job-cuts-across-departments.html',
    'summary': 'Microsoft has confirmed it is cutting a small percentage of jobs across departments, affecting less than 1% of its 228,000 employees, as part of a performance-based restructuring effort, despite reporting a net income margin of nearly 38% and a market capitalization of over $3 trillion.',
    'date': '2025-01-09',
    'layoffs': '228',
    'reason': 'performance-based restructuring effort',
    'liquidity': np.float64(3.2),
    'sentiment': -0.9992722868919373},
   {'ticker': 'microsoft',
    'title': 'Microsoft confirms more job cuts on top of 10,000 layoffs in January',
    'url': 'https://www.cnbc.com/2023/07/10/microsoft-confirms-more-job-cuts-on-top-of-10000-layoffs-in-january.html',
    'summary': 'Microsoft has confirmed additional job cuts, following its January announcement of 10,000 layoffs, with the company eliminating an undisclosed number of positions, including 276 in its home state of Washington, as it adjusts to changing market conditions and client demands for cost savings.',
    'date': '2023-07-11',
    'layoffs': '10,000',
    'reason': 'adjusting to changing market conditions and client demands for cost savings',
    'liquidity': np.float64(-7.3),
    'sentiment': -0.9984258413314819},
   {'ticker': 'microsoft',
    'title': "U.S. job market divide boosts some workers' prospects, puts others ...",
    'url': 'https://www.cnbc.com/2022/05/29/us-job-market-divide-boosts-some-workers-prospects-puts-others-on-notice.html',
    'summary': 'A diverging US job market is emerging, with some companies, including Microsoft, Twitter, and Meta, planning to curb hiring due to economic uncertainty, while others, such as airlines and restaurants, are struggling to fill positions, resulting in a record 11.55 million job openings and a potential slowdown in wage growth and consumer spending.',
    'date': '2022-05-29',
    'layoffs': '0',
    'reason': 'Economic uncertainty',
    'liquidity': np.float64(28.2),
    'sentiment': -0.9487793445587158}]},
 {'apple': [{'ticker': 'apple',
    'title': 'US Forest Service fires 3,400 workers, Park Service cuts 1,000 ...',
    'url': 'https://www.reuters.com/world/us/us-forest-service-fires-3400-workers-park-service-cuts-1000-2025-02-14/',
    'summary': 'The US Forest Service has fired approximately 3,400 workers, while the National Park Service has cut around 1,000 jobs, as part of a broader effort to reduce costs and streamline operations, with the latter move expected to save around $150 million annually.',
    'date': '2025-02-14',
    'layoffs': '3,400',
    'reason': 'reduce costs and streamline operations',
    'liquidity': np.float64(-41.3),
    'sentiment': -0.9882815480232239},
   {'ticker': 'apple',
    'title': 'A comprehensive list of 2024 and 2025 tech layoffs | TechCrunch',
    'url': 'https://techcrunch.com/2025/02/13/tech-layoffs-2024-list/',
    'summary': 'Here is a concise news summary:\n\nThe tech industry has seen a significant wave of layoffs in 2025, with over 22,000 workers cut from their jobs across 150 companies, including major players like Microsoft, Amazon, and Google, with the majority of cuts occurring in February, totaling 16,084 layoffs.',
    'date': '2025-02-13',
    'layoffs': '22,000',
    'reason': 'Significant wave of layoffs in 2025',
    'liquidity': np.float64(-21.8),
    'sentiment': -0.9499928951263428},
   {'ticker': 'apple',
    'title': 'Apple cuts about 100 digital services jobs, Bloomberg News reports ...',
    'url': 'https://www.reuters.com/technology/apple-cuts-about-100-jobs-its-services-division-bloomberg-news-reports-2024-08-28/',
    'summary': 'Apple has cut around 100 jobs from its digital services division, according to Bloomberg News, while the US is reportedly pushing Vietnam to reduce its reliance on Chinese technology in devices assembled in the country for export to America, as part of ongoing tariff talks.',
    'date': '2024-08-28',
    'layoffs': '100',
    'reason': 'Cutting jobs from digital services division',
    'liquidity': np.float64(39.5),
    'sentiment': -0.996970534324646},
   {'ticker': 'apple',
    'title': 'Apple agrees to $25 million settlement with US over hiring of ...',
    'url': 'https://www.reuters.com/technology/apple-agrees-25-million-settlement-with-us-over-hiring-immigrants-2023-11-09/',
    'summary': 'Apple has agreed to a $25 million settlement with the US government over allegations that the tech giant hired hundreds of foreign workers in the US without properly verifying their immigration status, marking a significant financial blow to the company.',
    'date': '2023-11-09',
    'layoffs': '0',
    'reason': 'Hiring foreign workers without proper verification',
    'liquidity': np.float64(-14.7),
    'sentiment': -0.9995536208152771},
   {'ticker': 'apple',
    'title': "Apple layoffs are a 'last resort,' CEO Tim Cook says",
    'url': 'https://www.cnbc.com/2023/05/04/apple-layoffs-are-a-last-resort-tim-cook-says.html',
    'summary': 'Apple CEO Tim Cook has ruled out large-scale layoffs, stating that mass layoffs are a "last resort" and not currently being considered, despite the company slowing its hiring rate and cutting costs, with Apple reporting a net income of $24 billion on $95 billion in revenue during its March quarter earnings report.',
    'date': '2023-05-04',
    'layoffs': '0',
    'reason': 'not considering mass layoffs',
    'liquidity': np.float64(7.7),
    'sentiment': -0.996056318283081},
   {'ticker': 'apple',
    'title': 'Apple delaying some employee bonuses as the company cuts costs ...',
    'url': 'https://www.cnbc.com/2023/03/14/apple-delaying-some-employee-bonuses-as-the-company-cuts-costs-report.html',
    'summary': 'Apple is reportedly delaying bonuses for some employees and implementing cost-cutting measures, including reducing travel budgets and leaving some positions unfilled, as the company navigates a tough market and slows hiring in non-engineering divisions, despite not having laid off employees.',
    'date': '2023-03-14',
    'layoffs': '0',
    'reason': 'not having laid off employees',
    'liquidity': np.float64(1.5),
    'sentiment': -0.9995632767677307},
   {'ticker': 'apple',
    'title': 'Apple grew more slowly than Google, Amazon, Microsoft and Meta',
    'url': 'https://www.cnbc.com/2023/01/18/apple-had-slower-headcount-growth-than-tech-peers-no-layoffs-yet.html',
    'summary': 'Here is a concise news summary:\n\nApple, the only major tech company to avoid significant layoffs, has maintained a steady workforce size, unlike its peers Google, Amazon, Microsoft, and Meta, which have collectively cut over 41,000 jobs, citing macroeconomic conditions and the possibility of a recession, with Microsoft announcing the largest layoff, cutting 10,000 employees, or 5% of its workforce.',
    'date': '2023-01-18',
    'layoffs': '41,000',
    'reason': 'macroeconomic conditions and the possibility of a recession',
    'liquidity': np.float64(20.4),
    'sentiment': 0.9708353877067566},
   {'ticker': 'apple',
    'title': 'Apple hires ex-Google AI scientist who resigned after colleagues ...',
    'url': 'https://www.reuters.com/technology/apple-hires-ex-google-ai-scientist-who-resigned-after-colleagues-firing-2021-05-03/',
    'summary': 'Apple has hired a former Google AI scientist, who resigned after colleagues were fired, as the tech giant continues to bolster its artificial intelligence capabilities, while the US is reportedly pushing Vietnam to reduce its reliance on Chinese technology in devices assembled for export to America.',
    'date': '2021-05-03',
    'layoffs': '0',
    'reason': 'No layoffs mentioned',
    'liquidity': np.float64(-4.7),
    'sentiment': 0.9066983461380005}]},
 {'amazon': [{'ticker': 'amazon',
    'title': 'Exclusive: Amazon trims jobs in devices and services unit | Reuters',
    'url': 'https://www.reuters.com/business/world-at-work/amazon-trims-jobs-devices-services-unit-2025-05-14/',
    'summary': "Here is a concise news summary:\n\nAmazon has trimmed jobs in its devices and services unit, while the UK's competition watchdog launched a probe into the proposed $13.25 billion merger between advertising giants Omnicom and Interpublic, citing concerns over reduced competition in the UK market.",
    'date': '2025-05-14',
    'layoffs': '10,000',
    'reason': 'trimmed jobs in devices and services unit',
    'liquidity': np.float64(-13.4),
    'sentiment': -0.989094078540802},
   {'ticker': 'amazon',
    'title': 'Amazon cuts some jobs in its communications, sustainability units',
    'url': 'https://www.cnbc.com/2025/01/29/amazon-cuts-some-jobs-in-its-communications-sustainability-units.html',
    'summary': 'Amazon has laid off a "small number" of employees in its communications and sustainability units, as part of a wider review of its organizational design, with the company providing financial support, continued benefits, and job-placement assistance to those affected.',
    'date': '2025-01-29',
    'layoffs': 'a small number',
    'reason': 'part of a wider review of its organizational design',
    'liquidity': np.float64(-16.9),
    'sentiment': -0.9250519871711731},
   {'ticker': 'amazon',
    'title': 'Amazon layoffs: Hundreds of jobs cut in cloud computing unit',
    'url': 'https://www.cnbc.com/2024/04/03/amazon-layoffs-hundreds-of-jobs-cut-in-cloud-computing-unit.html',
    'summary': "Amazon's cloud computing division, Amazon Web Services (AWS), is cutting hundreds of jobs in its physical stores technology and sales and marketing units, citing a need to streamline operations and focus on key strategic areas, as the company continues to trim its headcount after a year of mass layoffs.",
    'date': '2024-04-03',
    'layoffs': 'hundreds',
    'reason': 'streamline operations and focus on key strategic areas',
    'liquidity': np.float64(36.9),
    'sentiment': -0.9986535310745239},
   {'ticker': 'amazon',
    'title': 'Amazon cuts hundreds of jobs in Pharmacy, One Medical units ...',
    'url': 'https://www.cnbc.com/2024/02/06/amazon-cuts-hundreds-of-jobs-cut-in-pharmacy-one-medical-read-memo.html',
    'summary': "Amazon has eliminated hundreds of jobs in its Pharmacy and One Medical divisions, citing a need to reposition resources and invest in areas that directly impact customers, as part of the company's ongoing efforts to trim its headcount and cut costs.",
    'date': '2024-02-06',
    'layoffs': 'hundreds',
    'reason': 'reposition resources and invest in areas that directly impact customers',
    'liquidity': np.float64(-33.9),
    'sentiment': -0.999089241027832},
   {'ticker': 'amazon',
    'title': 'Amazon layoffs: Hundreds of jobs cut in Prime Video and MGM ...',
    'url': 'https://www.cnbc.com/2024/01/10/amazon-layoffs-hundreds-of-jobs-cut-in-prime-video-and-mgm-studios.html',
    'summary': 'Amazon has laid off hundreds of employees across its Prime Video and MGM Studios divisions, citing a need to prioritize investments for long-term success, as part of a broader effort to trim its headcount after a year of mass layoffs, which have already resulted in over 27,000 job cuts across the company.',
    'date': '2024-01-10',
    'layoffs': 'hundreds',
    'reason': 'prioritize investments for long-term success',
    'liquidity': np.float64(-0.2),
    'sentiment': -0.9987525939941406},
   {'ticker': 'amazon',
    'title': "Amazon cuts 'several hundred' jobs in Alexa division",
    'url': 'https://www.cnbc.com/2023/11/17/amazon-cuts-several-hundred-jobs-in-alexa-division.html',
    'summary': 'Amazon has begun laying off "several hundred" employees in its Alexa division, as part of a broader cost-cutting effort, with the company citing a need to "maximize resources and efforts focused on generative AI" and discontinuing some initiatives, despite Alexa\'s continued popularity with over 500 million devices in use.',
    'date': '2023-11-17',
    'layoffs': 'several hundred',
    'reason': 'maximize resources and efforts focused on generative AI',
    'liquidity': np.float64(-19.0),
    'sentiment': -0.9995208978652954},
   {'ticker': 'amazon',
    'title': 'Amazon layoffs: Company to cut 9,000 more workers',
    'url': 'https://www.cnbc.com/2023/03/20/amazon-layoffs-company-to-cut-off-9000-more-workers.html',
    'summary': 'Amazon will lay off an additional 9,000 employees, on top of the 18,000 cuts announced earlier, as the company seeks to streamline costs and adapt to economic uncertainty, with the latest round primarily impacting its cloud computing, human resources, advertising, and Twitch livestreaming businesses.',
    'date': '2023-03-20',
    'layoffs': '27,000',
    'reason': 'streamlining costs and adapting to economic uncertainty',
    'liquidity': np.float64(-24.0),
    'sentiment': -0.9940645098686218},
   {'ticker': 'amazon',
    'title': 'Amazon to cut 18,000 jobs as layoffs expand in tech sector | Reuters',
    'url': 'https://www.reuters.com/business/retail-consumer/amazon-lay-off-over-17000-workers-wsj-2023-01-04/',
    'summary': 'Amazon is cutting 18,000 jobs, expanding the tech sector\'s layoffs, as the company\'s CEO Andy Jassy cites a need to "right-size" the business, while the UK\'s competition watchdog launches a probe into the proposed $13.25 billion merger between Omnicom and Interpublic, a deal that could lead to reduced competition in the UK market.',
    'date': '2023-01-05',
    'layoffs': '18,000',
    'reason': 'to right-size the business',
    'liquidity': np.float64(16.6),
    'sentiment': -0.8937264680862427}]}]

In [ ]:
liquidity_data=[{'meta': [{'Avg_Volume': 36963641.666666664,
    'Avg_Spread': 6.951403071879359,
    'date': '2022-06'},
   {'Avg_Volume': 29324120.0,
    'Avg_Spread': 6.160465369046006,
    'date': '2022-07'},
   {'Avg_Volume': 23996647.826086957,
    'Avg_Spread': 5.4369853908219605,
    'date': '2022-08'},
   {'Avg_Volume': 30238800.0,
    'Avg_Spread': 5.384621519807248,
    'date': '2022-09'},
   {'Avg_Volume': 52745819.047619045,
    'Avg_Spread': 5.486048870606757,
    'date': '2022-10'},
   {'Avg_Volume': 50656723.809523806,
    'Avg_Spread': 3.9627518171633396,
    'date': '2022-11'},
   {'Avg_Volume': 31096180.95238095,
    'Avg_Spread': 3.9314708242333247,
    'date': '2022-12'},
   {'Avg_Volume': 27438530.0,
    'Avg_Spread': 4.141978969653569,
    'date': '2023-01'},
   {'Avg_Volume': 40640573.684210524,
    'Avg_Spread': 6.415551396443679,
    'date': '2023-02'},
   {'Avg_Volume': 30002304.347826086,
    'Avg_Spread': 5.967527587234378,
    'date': '2023-03'},
   {'Avg_Volume': 23509889.47368421,
    'Avg_Spread': 4.551180465083902,
    'date': '2023-04'},
   {'Avg_Volume': 22134931.818181816,
    'Avg_Spread': 6.067311015704079,
    'date': '2023-05'},
   {'Avg_Volume': 22903804.76190476,
    'Avg_Spread': 7.631176131089457,
    'date': '2023-06'},
   {'Avg_Volume': 31230255.0,
    'Avg_Spread': 9.118029770957094,
    'date': '2023-07'},
   {'Avg_Volume': 18397730.43478261,
    'Avg_Spread': 8.24159514539258,
    'date': '2023-08'},
   {'Avg_Volume': 20334330.0,
    'Avg_Spread': 8.414342437973175,
    'date': '2023-09'},
   {'Avg_Volume': 23241268.181818184,
    'Avg_Spread': 8.882678312806782,
    'date': '2023-10'},
   {'Avg_Volume': 15679547.61904762,
    'Avg_Spread': 6.8382460613994756,
    'date': '2023-11'},
   {'Avg_Volume': 16640690.0,
    'Avg_Spread': 7.142834233255431,
    'date': '2023-12'},
   {'Avg_Volume': 16524771.42857143,
    'Avg_Spread': 8.072905598316408,
    'date': '2024-01'},
   {'Avg_Volume': 23307570.0,
    'Avg_Spread': 11.290648776975036,
    'date': '2024-02'},
   {'Avg_Volume': 13977945.0,
    'Avg_Spread': 12.305606634922535,
    'date': '2024-03'},
   {'Avg_Volume': 19869627.272727273,
    'Avg_Spread': 15.601724781745686,
    'date': '2024-04'},
   {'Avg_Volume': 12839763.636363637,
    'Avg_Spread': 10.11220029289412,
    'date': '2024-05'},
   {'Avg_Volume': 11944784.210526315,
    'Avg_Spread': 10.57269959179415,
    'date': '2024-06'},
   {'Avg_Volume': 14727518.181818182,
    'Avg_Spread': 14.075387064644671,
    'date': '2024-07'},
   {'Avg_Volume': 14699054.545454545,
    'Avg_Spread': 14.974821941226985,
    'date': '2024-08'},
   {'Avg_Volume': 12321900.0,
    'Avg_Spread': 13.017264881944673,
    'date': '2024-09'},
   {'Avg_Volume': 11559926.086956521,
    'Avg_Spread': 12.060831423027851,
    'date': '2024-10'},
   {'Avg_Volume': 11890560.0,
    'Avg_Spread': 12.863165542442545,
    'date': '2024-11'},
   {'Avg_Volume': 12534414.285714285,
    'Avg_Spread': 15.49284707254678,
    'date': '2024-12'},
   {'Avg_Volume': 15897110.0,
    'Avg_Spread': 19.41172461488132,
    'date': '2025-01'},
   {'Avg_Volume': 15706247.368421054,
    'Avg_Spread': 19.279170656455598,
    'date': '2025-02'},
   {'Avg_Volume': 17305404.76190476,
    'Avg_Spread': 22.463572640658565,
    'date': '2025-03'},
   {'Avg_Volume': 21493895.23809524,
    'Avg_Spread': 26.08856927780878,
    'date': '2025-04'},
   {'Avg_Volume': 13927595.238095239,
    'Avg_Spread': 14.687136695498513,
    'date': '2025-05'},
   {'Avg_Volume': 11598390.0,
    'Avg_Spread': 14.478997802734375,
    'date': '2025-06'}]},
 {'google': [{'Avg_Volume': 38689833.333333336,
    'Avg_Spread': 3.5248603082778374,
    'date': '2022-06'},
   {'Avg_Volume': 39476485.0,
    'Avg_Spread': 3.7984440283774434,
    'date': '2022-07'},
   {'Avg_Volume': 22428378.260869566,
    'Avg_Spread': 2.374887288754786,
    'date': '2022-08'},
   {'Avg_Volume': 29203757.14285714,
    'Avg_Spread': 2.6337276805278966,
    'date': '2022-09'},
   {'Avg_Volume': 32451823.80952381,
    'Avg_Spread': 2.9111117698971603,
    'date': '2022-10'},
   {'Avg_Volume': 34120128.571428575,
    'Avg_Spread': 2.8992794889703695,
    'date': '2022-11'},
   {'Avg_Volume': 28720371.42857143,
    'Avg_Spread': 2.331730204790997,
    'date': '2022-12'},
   {'Avg_Volume': 33644890.0,
    'Avg_Spread': 2.4517973668487443,
    'date': '2023-01'},
   {'Avg_Volume': 50135800.0,
    'Avg_Spread': 2.8214995661982734,
    'date': '2023-02'},
   {'Avg_Volume': 37388430.43478261,
    'Avg_Spread': 2.564186997888298,
    'date': '2023-03'},
   {'Avg_Volume': 31795068.42105263,
    'Avg_Spread': 2.475156721296521,
    'date': '2023-04'},
   {'Avg_Volume': 37297518.18181818,
    'Avg_Spread': 2.8108687758591016,
    'date': '2023-05'},
   {'Avg_Volume': 31274128.57142857,
    'Avg_Spread': 2.786147694555308,
    'date': '2023-06'},
   {'Avg_Volume': 35417075.0,
    'Avg_Spread': 2.720185898687047,
    'date': '2023-07'},
   {'Avg_Volume': 25795173.913043477,
    'Avg_Spread': 2.6601320100753805,
    'date': '2023-08'},
   {'Avg_Volume': 23877105.0,
    'Avg_Spread': 2.5169044780143692,
    'date': '2023-09'},
   {'Avg_Volume': 30941490.90909091,
    'Avg_Spread': 2.830300144231478,
    'date': '2023-10'},
   {'Avg_Volume': 25486680.95238095,
    'Avg_Spread': 2.2862863697891305,
    'date': '2023-11'},
   {'Avg_Volume': 30995150.0,
    'Avg_Spread': 2.3280366067917564,
    'date': '2023-12'},
   {'Avg_Volume': 27660714.285714287,
    'Avg_Spread': 2.5816609325308013,
    'date': '2024-01'},
   {'Avg_Volume': 32383470.0,
    'Avg_Spread': 2.646131258089653,
    'date': '2024-02'},
   {'Avg_Volume': 33630295.0,
    'Avg_Spread': 2.825055471324373,
    'date': '2024-03'},
   {'Avg_Volume': 29975954.545454547,
    'Avg_Spread': 3.495854944988197,
    'date': '2024-04'},
   {'Avg_Volume': 24654854.545454547,
    'Avg_Spread': 2.9902479038572225,
    'date': '2024-05'},
   {'Avg_Volume': 24267910.52631579,
    'Avg_Spread': 2.9931349975006785,
    'date': '2024-06'},
   {'Avg_Volume': 23914922.727272727,
    'Avg_Spread': 3.6188329117501827,
    'date': '2024-07'},
   {'Avg_Volume': 25033613.636363637,
    'Avg_Spread': 3.6898515636531823,
    'date': '2024-08'},
   {'Avg_Volume': 26630830.0,
    'Avg_Spread': 3.1401999046774662,
    'date': '2024-09'},
   {'Avg_Volume': 24752569.56521739,
    'Avg_Spread': 3.421029484856071,
    'date': '2024-10'},
   {'Avg_Volume': 26499530.0,
    'Avg_Spread': 3.325819704610164,
    'date': '2024-11'},
   {'Avg_Volume': 30743342.85714286,
    'Avg_Spread': 4.6063272597632094,
    'date': '2024-12'},
   {'Avg_Volume': 25018775.0,
    'Avg_Spread': 4.1826123726620255,
    'date': '2025-01'},
   {'Avg_Volume': 33125100.0,
    'Avg_Spread': 3.970088030173605,
    'date': '2025-02'},
   {'Avg_Volume': 34864395.23809524,
    'Avg_Spread': 4.75648174116757,
    'date': '2025-03'},
   {'Avg_Volume': 40120961.9047619,
    'Avg_Spread': 5.515229746615895,
    'date': '2025-04'},
   {'Avg_Volume': 44883366.666666664,
    'Avg_Spread': 4.684806904262866,
    'date': '2025-05'},
   {'Avg_Volume': 35364070.0,
    'Avg_Spread': 3.3162399882162843,
    'date': '2025-06'}]},
 {'microsoft': [{'Avg_Volume': 29781775.0,
    'Avg_Spread': 6.214855754216349,
    'date': '2022-06'},
   {'Avg_Volume': 26744590.0,
    'Avg_Spread': 6.901436040656509,
    'date': '2022-07'},
   {'Avg_Volume': 20745982.608695652,
    'Avg_Spread': 4.776837602288796,
    'date': '2022-08'},
   {'Avg_Volume': 27408885.714285713,
    'Avg_Spread': 5.8574721838056645,
    'date': '2022-09'},
   {'Avg_Volume': 31963100.0,
    'Avg_Spread': 6.917180930215765,
    'date': '2022-10'},
   {'Avg_Volume': 29299809.523809522,
    'Avg_Spread': 6.114430705298725,
    'date': '2022-11'},
   {'Avg_Volume': 28159938.095238097,
    'Avg_Spread': 5.817509508868594,
    'date': '2022-12'},
   {'Avg_Volume': 33308405.0,
    'Avg_Spread': 5.527194990542002,
    'date': '2023-01'},
   {'Avg_Volume': 32394789.47368421,
    'Avg_Spread': 5.760960765741213,
    'date': '2023-02'},
   {'Avg_Volume': 32505869.56521739,
    'Avg_Spread': 6.008022906416483,
    'date': '2023-03'},
   {'Avg_Volume': 29026163.157894738,
    'Avg_Spread': 5.476886733076021,
    'date': '2023-04'},
   {'Avg_Volume': 27309418.181818184,
    'Avg_Spread': 4.969685802446755,
    'date': '2023-05'},
   {'Avg_Volume': 26075652.38095238,
    'Avg_Spread': 6.3052858673515555,
    'date': '2023-06'},
   {'Avg_Volume': 33338220.0,
    'Avg_Spread': 8.064105105489787,
    'date': '2023-07'},
   {'Avg_Volume': 20845943.47826087,
    'Avg_Spread': 5.55459072396829,
    'date': '2023-08'},
   {'Avg_Volume': 20834035.0,
    'Avg_Spread': 5.4154122592086456,
    'date': '2023-09'},
   {'Avg_Volume': 24586681.818181816,
    'Avg_Spread': 6.9491318223332685,
    'date': '2023-10'},
   {'Avg_Volume': 26851442.85714286,
    'Avg_Spread': 5.30619809612139,
    'date': '2023-11'},
   {'Avg_Volume': 26100185.0,
    'Avg_Spread': 5.056107572523127,
    'date': '2023-12'},
   {'Avg_Volume': 25161857.14285714,
    'Avg_Spread': 6.381374205222734,
    'date': '2024-01'},
   {'Avg_Volume': 22208815.0,
    'Avg_Spread': 5.692849090275558,
    'date': '2024-02'},
   {'Avg_Volume': 21317480.0,
    'Avg_Spread': 6.342345424550885,
    'date': '2024-03'},
   {'Avg_Volume': 20035350.0,
    'Avg_Spread': 7.201250302024632,
    'date': '2024-04'},
   {'Avg_Volume': 18809127.272727273,
    'Avg_Spread': 6.517289904349494,
    'date': '2024-05'},
   {'Avg_Volume': 18019494.736842107,
    'Avg_Spread': 6.111043228783726,
    'date': '2024-06'},
   {'Avg_Volume': 20020418.181818184,
    'Avg_Spread': 8.384356406159146,
    'date': '2024-07'},
   {'Avg_Volume': 20541772.727272727,
    'Avg_Spread': 7.346681323524006,
    'date': '2024-08'},
   {'Avg_Volume': 18846060.0,
    'Avg_Spread': 7.141690302712917,
    'date': '2024-09'},
   {'Avg_Volume': 19162847.826086957,
    'Avg_Spread': 6.509718547548311,
    'date': '2024-10'},
   {'Avg_Volume': 22116060.0,
    'Avg_Spread': 7.447362948017738,
    'date': '2024-11'},
   {'Avg_Volume': 20947733.333333332,
    'Avg_Spread': 7.432378566955252,
    'date': '2024-12'},
   {'Avg_Volume': 23128135.0,
    'Avg_Spread': 8.157135129782764,
    'date': '2025-01'},
   {'Avg_Volume': 22760468.42105263,
    'Avg_Spread': 6.77094953371524,
    'date': '2025-02'},
   {'Avg_Volume': 23418409.523809522,
    'Avg_Spread': 8.480617608408942,
    'date': '2025-03'},
   {'Avg_Volume': 26807980.95238095,
    'Avg_Spread': 12.618736360368668,
    'date': '2025-04'},
   {'Avg_Volume': 22711542.85714286,
    'Avg_Spread': 6.674318771393375,
    'date': '2025-05'},
   {'Avg_Volume': 16595280.0,
    'Avg_Spread': 5.11300048828125,
    'date': '2025-06'}]},
 {'apple': [{'Avg_Volume': 86451666.66666667,
    'Avg_Spread': 3.6186095331209835,
    'date': '2022-06'},
   {'Avg_Volume': 72356270.0,
    'Avg_Spread': 3.5672688384329403,
    'date': '2022-07'},
   {'Avg_Volume': 65662591.30434783,
    'Avg_Spread': 3.010744593886386,
    'date': '2022-08'},
   {'Avg_Volume': 99272514.28571428,
    'Avg_Spread': 4.351559300716413,
    'date': '2022-09'},
   {'Avg_Volume': 88959033.33333333,
    'Avg_Spread': 4.257694459443509,
    'date': '2022-10'},
   {'Avg_Volume': 82135604.76190476,
    'Avg_Spread': 4.465533022298607,
    'date': '2022-11'},
   {'Avg_Volume': 79796723.8095238,
    'Avg_Spread': 3.8397840406626926,
    'date': '2022-12'},
   {'Avg_Volume': 72182625.0,
    'Avg_Spread': 3.489299040870771,
    'date': '2023-01'},
   {'Avg_Volume': 68799942.10526316,
    'Avg_Spread': 3.2456567892878954,
    'date': '2023-02'},
   {'Avg_Volume': 66098547.82608695,
    'Avg_Spread': 3.0865254455370867,
    'date': '2023-03'},
   {'Avg_Volume': 51037352.631578945,
    'Avg_Spread': 2.3463878589065215,
    'date': '2023-04'},
   {'Avg_Volume': 57961613.63636363,
    'Avg_Spread': 2.2838791139200705,
    'date': '2023-05'},
   {'Avg_Volume': 61766719.047619045,
    'Avg_Spread': 2.794402776782582,
    'date': '2023-06'},
   {'Avg_Volume': 49803320.0,
    'Avg_Spread': 2.680662451966066,
    'date': '2023-07'},
   {'Avg_Volume': 57497365.217391305,
    'Avg_Spread': 3.0675047515113683,
    'date': '2023-08'},
   {'Avg_Volume': 66879330.0,
    'Avg_Spread': 3.384228124658854,
    'date': '2023-09'},
   {'Avg_Volume': 53305436.36363637,
    'Avg_Spread': 3.0328582237653294,
    'date': '2023-10'},
   {'Avg_Volume': 52361242.85714286,
    'Avg_Spread': 2.3401329096017087,
    'date': '2023-11'},
   {'Avg_Volume': 53138740.0,
    'Avg_Spread': 2.402903400177651,
    'date': '2023-12'},
   {'Avg_Volume': 56534252.38095238,
    'Avg_Spread': 2.878444584186675,
    'date': '2024-01'},
   {'Avg_Volume': 58081350.0,
    'Avg_Spread': 2.9925379492893582,
    'date': '2024-02'},
   {'Avg_Volume': 71639140.0,
    'Avg_Spread': 3.038649928182383,
    'date': '2024-03'},
   {'Avg_Volume': 56623500.0,
    'Avg_Spread': 2.827015025374635,
    'date': '2024-04'},
   {'Avg_Volume': 60751713.63636363,
    'Avg_Spread': 2.750781697344377,
    'date': '2024-05'},
   {'Avg_Volume': 90736026.31578948,
    'Avg_Spread': 5.092500842522816,
    'date': '2024-06'},
   {'Avg_Volume': 52413627.27272727,
    'Avg_Spread': 4.619309068707114,
    'date': '2024-07'},
   {'Avg_Volume': 51030318.18181818,
    'Avg_Spread': 4.957191499110639,
    'date': '2024-08'},
   {'Avg_Volume': 61607015.0,
    'Avg_Spread': 4.118024310970401,
    'date': '2024-09'},
   {'Avg_Volume': 40466782.60869565,
    'Avg_Spread': 3.5566269774208625,
    'date': '2024-10'},
   {'Avg_Volume': 44582030.0,
    'Avg_Spread': 3.437653175890719,
    'date': '2024-11'},
   {'Avg_Volume': 46567433.333333336,
    'Avg_Spread': 3.928144886103761,
    'date': '2024-12'},
   {'Avg_Volume': 60014585.0,
    'Avg_Spread': 5.720699524549056,
    'date': '2025-01'},
   {'Avg_Volume': 45382752.631578945,
    'Avg_Spread': 5.256802438480764,
    'date': '2025-02'},
   {'Avg_Volume': 53106642.85714286,
    'Avg_Spread': 6.185698256141528,
    'date': '2025-03'},
   {'Avg_Volume': 76499438.09523809,
    'Avg_Spread': 9.0200763085349,
    'date': '2025-04'},
   {'Avg_Volume': 56939438.0952381,
    'Avg_Spread': 4.3773694163750765,
    'date': '2025-05'},
   {'Avg_Volume': 51093480.0,
    'Avg_Spread': 4.004998779296875,
    'date': '2025-06'}]},
 {'amazon': [{'Avg_Volume': 75248608.33333333,
    'Avg_Spread': 5.198333104451497,
    'date': '2022-06'},
   {'Avg_Volume': 66892630.0,
    'Avg_Spread': 4.354499435424804,
    'date': '2022-07'},
   {'Avg_Volume': 50889086.95652174,
    'Avg_Spread': 3.8026103144106655,
    'date': '2022-08'},
   {'Avg_Volume': 57642266.666666664,
    'Avg_Spread': 3.8761912754603793,
    'date': '2022-09'},
   {'Avg_Volume': 69491023.8095238,
    'Avg_Spread': 4.44857170468285,
    'date': '2022-10'},
   {'Avg_Volume': 96911104.76190476,
    'Avg_Spread': 4.024285816010975,
    'date': '2022-11'},
   {'Avg_Volume': 73771109.52380952,
    'Avg_Spread': 2.661904834565662,
    'date': '2022-12'},
   {'Avg_Volume': 76189930.0,
    'Avg_Spread': 3.3145000457763674,
    'date': '2023-01'},
   {'Avg_Volume': 71794842.10526316,
    'Avg_Spread': 3.155263398822985,
    'date': '2023-02'},
   {'Avg_Volume': 58662621.73913044,
    'Avg_Spread': 2.887391131857167,
    'date': '2023-03'},
   {'Avg_Volume': 64425452.631578945,
    'Avg_Spread': 2.926842538934005,
    'date': '2023-04'},
   {'Avg_Volume': 65131436.36363637,
    'Avg_Spread': 2.741818514737216,
    'date': '2023-05'},
   {'Avg_Volume': 59173752.38095238,
    'Avg_Spread': 3.1785714285714284,
    'date': '2023-06'},
   {'Avg_Volume': 52937740.0,
    'Avg_Spread': 2.731496810913086,
    'date': '2023-07'},
   {'Avg_Volume': 52627226.08695652,
    'Avg_Spread': 2.923476841138757,
    'date': '2023-08'},
   {'Avg_Volume': 56013595.0, 'Avg_Spread': 3.35, 'date': '2023-09'},
   {'Avg_Volume': 55662031.81818182,
    'Avg_Spread': 3.5004553361372515,
    'date': '2023-10'},
   {'Avg_Volume': 48856519.047619045,
    'Avg_Spread': 2.7219027564639138,
    'date': '2023-11'},
   {'Avg_Volume': 46556430.0,
    'Avg_Spread': 2.739000701904297,
    'date': '2023-12'},
   {'Avg_Volume': 45397376.190476194,
    'Avg_Spread': 2.9404754638671875,
    'date': '2024-01'},
   {'Avg_Volume': 52253060.0,
    'Avg_Spread': 3.205000305175781,
    'date': '2024-02'},
   {'Avg_Volume': 35096445.0,
    'Avg_Spread': 3.191998291015625,
    'date': '2024-03'},
   {'Avg_Volume': 41682777.27272727,
    'Avg_Spread': 4.274546536532315,
    'date': '2024-04'},
   {'Avg_Volume': 40559168.18181818,
    'Avg_Spread': 3.5749990289861504,
    'date': '2024-05'},
   {'Avg_Volume': 42804000.0,
    'Avg_Spread': 3.9421065481085527,
    'date': '2024-06'},
   {'Avg_Volume': 39457322.72727273,
    'Avg_Spread': 4.4122702858664775,
    'date': '2024-07'},
   {'Avg_Volume': 44137450.0,
    'Avg_Spread': 4.578636863014915,
    'date': '2024-08'},
   {'Avg_Volume': 38251440.0,
    'Avg_Spread': 4.003001403808594,
    'date': '2024-09'},
   {'Avg_Volume': 31907773.913043477,
    'Avg_Spread': 3.3226112697435464,
    'date': '2024-10'},
   {'Avg_Volume': 45296895.0,
    'Avg_Spread': 4.6069999694824215,
    'date': '2024-11'},
   {'Avg_Volume': 36232904.76190476,
    'Avg_Spread': 4.720476422991071,
    'date': '2024-12'},
   {'Avg_Volume': 32631460.0,
    'Avg_Spread': 4.862500762939453,
    'date': '2025-01'},
   {'Avg_Volume': 41242826.315789476,
    'Avg_Spread': 5.175790084035773,
    'date': '2025-02'},
   {'Avg_Volume': 46034371.428571425,
    'Avg_Spread': 6.210475376674107,
    'date': '2025-03'},
   {'Avg_Volume': 62515961.9047619,
    'Avg_Spread': 8.955239432198661,
    'date': '2025-04'},
   {'Avg_Volume': 44631557.14285714,
    'Avg_Spread': 4.352380661737351,
    'date': '2025-05'},
   {'Avg_Volume': 34953660.0,
    'Avg_Spread': 4.056001281738281,
    'date': '2025-06'}]}]

In [ ]:
insights_data=[{'meta': [{'ticker': 'meta',
    'title': 'In AI copyright case, Zuckerberg turns to YouTube for his defense ...',
    'url': 'https://techcrunch.com/2025/01/15/in-ai-copyright-case-zuckerberg-turns-to-youtube-for-his-defense/',
    'summary': "Meta CEO Mark Zuckerberg has defended his company's use of a copyrighted e-book dataset, LibGen, by citing YouTube's efforts to remove pirated content, stating that the vast majority of content on the platform is licensed and legitimate, in a deposition related to the AI copyright case Kadrey v. Meta Platforms.\n\nKey entities:\n\n* Mark Zuckerberg, Meta CEO\n* YouTube\n* Meta Platforms\n* LibGen, a copyrighted e-book dataset\n* Llama, Meta's AI models\n\nFinancial impacts and numbers:\n\n* None mentioned in the article.",
    'date': '2025-01-16',
    'labels': [],
    'sentiment': -0.994657039642334,
    'liquidity': np.float64(7.0)},
   {'ticker': 'meta',
    'title': 'Stainless helps build SDKs for OpenAI, Anthropic, and Meta ...',
    'url': 'https://techcrunch.com/2024/12/10/stainless-helps-build-sdks-for-openai-anthropic-and-meta/',
    'summary': 'Stainless, a company founded by Alex Rattray, has developed an AI-powered compiler that generates software development kits (SDKs) for APIs, partnering with OpenAI, Anthropic, and Meta to provide a scalable solution for companies to create apps using their products, with financial implications including increased efficiency and reduced costs for vendors.',
    'date': '2024-12-10',
    'labels': [],
    'sentiment': 0.9881121516227722,
    'liquidity': np.float64(-26.0)},
   {'ticker': 'meta',
    'title': 'Consumer Financial Protection Bureau moves to place Google ...',
    'url': 'https://techcrunch.com/2024/11/14/consumer-financial-protection-bureau-moves-to-place-google-under-supervision/',
    'summary': 'The Consumer Financial Protection Bureau (CFPB) has taken steps to place Google under formal federal supervision, a move that could subject the company to the same inspections as major banks, amid concerns over its financial practices and activities.',
    'date': '2024-11-14',
    'labels': [],
    'sentiment': -0.9972302317619324,
    'liquidity': np.float64(7.6)},
   {'ticker': 'meta',
    'title': "Meta's AI chief says world models are key to 'human-level AI' — but it ...",
    'url': 'https://techcrunch.com/2024/10/16/metas-ai-chief-says-world-models-are-key-to-human-level-ai-but-it-might-be-10-years-out/',
    'summary': 'Meta\'s AI chief, Yann LeCun, has stated that achieving "human-level AI" may take around 10 years, as current AI systems lack the ability to understand the world, remember things, and reason and plan like humans, despite advancements in large language models like ChatGPT.',
    'date': '2024-10-17',
    'labels': [],
    'sentiment': -0.9994743466377258,
    'liquidity': np.float64(-8.7)},
   {'ticker': 'meta',
    'title': "Meta's Hyperscape lets you scan and explore real-life spaces in VR ...",
    'url': 'https://techcrunch.com/2024/09/25/metas-hyperscape-lets-you-scan-and-explore-real-life-spaces-in-vr/',
    'summary': 'Meta has launched a new VR app called Hyperscape, which allows users to scan and explore real-life spaces in high fidelity using a Quest headset, with plans to add more features and compatibility with its VR platform Horizon, marking a significant step towards the metaverse.',
    'date': '2024-09-25',
    'labels': ['legal issues'],
    'sentiment': 0.9981163740158081,
    'liquidity': np.float64(-14.9)},
   {'ticker': 'meta',
    'title': 'Meta brings us a step closer to AI-generated movies | TechCrunch',
    'url': 'https://techcrunch.com/2023/11/16/meta-brings-us-a-step-closer-to-ai-generated-movies/',
    'summary': 'Meta has unveiled Emu Video, an AI-powered tool that can generate four-second animated clips from captions, images, or descriptions, marking a significant step towards the creation of AI-generated movies and TV shows, with potential financial impacts of increased efficiency and cost savings in the entertainment industry.',
    'date': '2023-11-16',
    'labels': [],
    'sentiment': 0.9978967905044556,
    'liquidity': np.float64(-10.2)},
   {'ticker': 'meta',
    'title': "As OpenAI's multimodal API launches broadly, research shows it's ...",
    'url': 'https://techcrunch.com/2023/11/06/openai-gpt-4-with-vision-release-research-flaws/',
    'summary': "OpenAI has launched its multimodal API, allowing developers to integrate its GPT-4 with vision AI model into their applications, which can understand the context of images and text, but research shows it still has flaws, despite OpenAI's confidence in its mitigations.",
    'date': '2023-11-06',
    'labels': [],
    'sentiment': -0.9992555975914001,
    'liquidity': np.float64(-28.6)},
   {'ticker': 'meta',
    'title': "Generative AI stickers are coming to Meta's apps along with AI ...",
    'url': 'https://techcrunch.com/2023/09/27/generative-ai-stickers-are-coming-to-metas-apps/',
    'summary': "Meta's CEO Mark Zuckerberg announced at the Meta Connect event that generative AI stickers will be rolled out across the company's messaging apps, including WhatsApp, Messenger, Instagram, and Facebook Stories, allowing users to create unique AI-generated stickers in seconds using the Emu foundational model for image generation.",
    'date': '2023-09-27',
    'labels': [],
    'sentiment': 0.9689184427261353,
    'liquidity': np.float64(21.0)},
   {'ticker': 'meta',
    'title': 'Harvard expert: New Meta return-to-office policy is a clear problem',
    'url': 'https://www.cnbc.com/2023/08/25/harvard-expert-new-meta-return-to-office-policy-is-a-clear-problem.html',
    'summary': 'Meta\'s new return-to-office mandate, requiring full-time employees to work three days a week in-office and track attendance, has sparked concerns about a "huge amount of distrust" among employees, with Harvard expert Heidi K. Gardner citing a lack of transparency and empathy from CEO Mark Zuckerberg.',
    'date': '2023-08-25',
    'labels': ['layoffs'],
    'sentiment': -0.9983364939689636,
    'liquidity': np.float64(-9.4)},
   {'ticker': 'meta',
    'title': 'What Facebook knew about how it radicalized users',
    'url': 'https://www.cnbc.com/2021/10/23/carols-journey-what-facebook-knew-about-how-it-radicalized-users.html',
    'summary': "Here is a concise news summary:\n\nFacebook's internal research revealed that its algorithm can radicalize users, including recommending conspiracy theory groups to a fictional user named Carol Smith, who was created as part of an experiment, with the company's own researchers warning that this can lead to millions of individuals being pushed into echo chambers and exposed to violent content.",
    'date': None,
    'labels': [],
    'sentiment': 0.9818291664123535,
    'liquidity': None}]},
 {'google': [{'ticker': 'google',
    'title': 'Google is shipping Gemini models faster than its AI safety reports ...',
    'url': 'https://techcrunch.com/2025/04/03/google-is-shipping-gemini-models-faster-than-its-ai-safety-reports/',
    'summary': 'Google has accelerated the release of its AI reasoning models, Gemini 2.5 Pro and Gemini 2.0 Flash, but has yet to publish safety reports, sparking concerns that the company is prioritizing speed over transparency, despite being a pioneer in promoting responsible AI practices.\n\n**Key entities:**\n\n* Google\n* OpenAI\n* Anthropic\n* Meta\n* Scale AI\n* Precursor Ventures\n* NEA\n* Index Ventures\n* Underscore VC\n\n**Financial impacts and numbers:**\n\n* No specific financial numbers mentioned in the article.',
    'date': '2025-04-03',
    'labels': [],
    'sentiment': -0.9969552755355835,
    'liquidity': np.float64(90.8)},
   {'ticker': 'google',
    'title': "Google scrubs mentions of 'diversity' and 'equity' from responsible AI ...",
    'url': 'https://techcrunch.com/2025/03/08/google-scrubs-mentions-of-diversity-and-equity-from-responsible-ai-team-webpage/',
    'summary': 'Google has quietly removed mentions of "diversity" and "equity" from the mission description of its Responsible AI and Human Centered Technology team, sparking concerns about the company\'s commitment to promoting inclusivity and fairness in its AI research and development.',
    'date': '2025-03-08',
    'labels': [],
    'sentiment': -0.9818302989006042,
    'liquidity': np.float64(-3.5)},
   {'ticker': 'google',
    'title': 'Google launches a free AI coding assistant with very high usage ...',
    'url': 'https://techcrunch.com/2025/02/25/google-launches-a-free-ai-coding-assistant-with-very-high-usage-caps/',
    'summary': 'Google has launched a free AI coding assistant, Gemini Code Assist, which allows developers to interact with a Google AI model in a chat window to access and edit their codebase, offering 180,000 code completions a month and 240 chat requests a day, significantly surpassing the usage caps of competing tools like GitHub Copilot.',
    'date': '2025-02-25',
    'labels': [],
    'sentiment': 0.9719516038894653,
    'liquidity': np.float64(29.4)},
   {'ticker': 'google',
    'title': 'Google puts $1M into 3D design app Rooms after more than 1 ...',
    'url': 'https://techcrunch.com/2025/01/07/google-puts-1m-into-3d-design-app-rooms-after-more-than-1-million-rooms-created/',
    'summary': 'Google has invested $1 million in 3D design app Rooms, which has seen over 1 million "rooms" created since its beta launch in 2023, allowing users to build and code interactive 3D spaces and mini-games using a library of over 10,000 items.',
    'date': '2025-01-07',
    'labels': [],
    'sentiment': 0.9595732092857361,
    'liquidity': np.float64(11.1)},
   {'ticker': 'google',
    'title': 'Google I/O was an AI evolution, not a revolution | TechCrunch',
    'url': 'https://techcrunch.com/2024/05/16/google-i-o-was-an-ai-evolution-not-a-revolution/',
    'summary': "At Google's I/O developer conference, the company showcased a range of AI-powered innovations, including a revamped search engine, AI helpers across its Workspace apps, and a future vision for AI, codenamed Project Astra, but the overwhelming amount of news left some consumers wondering what it all means and when they can expect to see these technologies in action.",
    'date': '2024-05-16',
    'labels': [],
    'sentiment': 0.9269447326660156,
    'liquidity': np.float64(-23.1)},
   {'ticker': 'google',
    'title': 'Facebook restricts users in Australia from sharing or viewing news ...',
    'url': 'https://techcrunch.com/2021/02/17/facebook-restricts-users-in-australia-from-sharing-or-viewing-news-links-in-response-to-proposed-legislation/',
    'summary': "Facebook has restricted Australian users from sharing or viewing news links on its platform, a move aimed at protesting the country's proposed legislation requiring tech giants to pay news publishers for content, with the company estimating that only 4% of users' feeds are comprised of news content.",
    'date': '2021-02-17',
    'labels': [],
    'sentiment': -0.9989079236984253,
    'liquidity': np.float64(9.8)},
   {'ticker': 'google',
    'title': 'Google Meet and other Google services go down (Updated ...',
    'url': 'https://techcrunch.com/2020/09/24/google-meet-and-other-google-services-go-down/',
    'summary': 'Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nGoogle\'s engineers faced a challenging day as multiple Google services, including Google Meet, Drive, Docs, Analytics, Classroom, and Calendar, went offline or became unreachable, affecting users worldwide, with no official statement on the cause of the outage, but a statement later confirming a "short service disruption" and recovery efforts.',
    'date': '2020-09-25',
    'labels': [],
    'sentiment': -0.7522803544998169,
    'liquidity': np.float64(-20.1)},
   {'ticker': 'google',
    'title': 'Google Goes After Reporters With New "Google Media Tools" Site ...',
    'url': 'https://techcrunch.com/2013/10/21/google-goes-after-reporters-with-new-google-media-tools-site/',
    'summary': 'Google has launched a new centralized resource, Google Media Tools, aimed at courting journalists, offering a suite of tools and resources to enhance their reporting, including access to Google Search trends, Google Maps, and details on how to use Google to cover various topics, with no financial impact reported.',
    'date': '2013-10-21',
    'labels': ['legal issues'],
    'sentiment': -0.9900816082954407,
    'liquidity': np.float64(-30.4)},
   {'ticker': 'google',
    'title': "Journalists' Addresses Posted In Revenge For Newspaper's Google ...",
    'url': 'https://techcrunch.com/2012/12/26/journalists-addresses-posted-google-maps-gun/',
    'summary': "Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\n**Summary:** A Connecticut lawyer, Christopher Fountain, posted the phone number and addresses of The Journal News' staff, including the publisher's home, in retaliation for the newspaper's publication of a Google Map showing the names and addresses of gun permit owners in select New York cities, sparking a transparency arms race and raising concerns about public safety.\n\n**Key Entities:**\n\n* The Journal News\n* Christopher Fountain\n* Google\n* Janet Hasson (The Journal's publisher)\n\n**Financial Impacts and Numbers:**\n\n* None mentioned in the article.",
    'date': '2012-12-26',
    'labels': [],
    'sentiment': 0.8795555233955383,
    'liquidity': np.float64(8.8)},
   {'ticker': 'google',
    'title': "Keen On... Why a Squirrel Dying on Your Front Lawn Isn't More ...",
    'url': 'https://techcrunch.com/2011/06/14/keen-on-eli-pariser-1/',
    'summary': 'Here is a concise news summary focusing on key entities, financial impacts, and numbers:\n\nGoogle is reportedly planning to cut ties with Scale AI, a company it acquired in 2021, amidst a $14.3 billion deal with Meta, while OpenAI releases a new AI reasoning model, o3-pro, and Waymo rides are found to be more expensive than Uber or Lyft, despite being popular.',
    'date': '2011-06-14',
    'labels': [],
    'sentiment': -0.9865429401397705,
    'liquidity': np.float64(76.7)}]},
 {'microsoft': [{'ticker': 'microsoft',
    'title': "Microsoft's relationship with OpenAI cracked when it hired Mustafa ...",
    'url': 'https://techcrunch.com/2025/01/22/microsofts-relationship-with-openai-cracked-when-it-hired-mustafa-suleyman-rival-marc-benioff-says/',
    'summary': "Microsoft's relationship with OpenAI has deteriorated, with Salesforce CEO Marc Benioff predicting that OpenAI will no longer rely on Microsoft as its exclusive cloud provider, citing the hiring of Mustafa Suleyman and the need for more data center capacity, which could potentially lead to OpenAI rivaling Microsoft in the future.",
    'date': '2025-01-22',
    'labels': [],
    'sentiment': -0.9989482760429382,
    'liquidity': np.float64(17.9)},
   {'ticker': 'microsoft',
    'title': 'US says Russian hackers stole federal government emails during ...',
    'url': 'https://techcrunch.com/2024/04/11/us-cisa-russia-apt-29-government-email-theft-microsoft/',
    'summary': 'The US Cybersecurity and Infrastructure Security Agency (CISA) has confirmed that Russian government-backed hackers, known as "Midnight Blizzard" or APT29, stole emails from several US federal agencies as a result of an ongoing cyberattack at Microsoft, compromising the security of federal government emails and presenting a "grave and unacceptable risk" to agencies.',
    'date': '2024-04-11',
    'labels': [],
    'sentiment': -0.9980389475822449,
    'liquidity': np.float64(46.8)},
   {'ticker': 'microsoft',
    'title': 'Microsoft employees exposed internal passwords in security lapse ...',
    'url': 'https://techcrunch.com/2024/04/09/microsoft-employees-exposed-internal-passwords-security-lapse/',
    'summary': 'Microsoft has resolved a security lapse that exposed internal company files and credentials to the open internet, including passwords, keys, and credentials used by employees to access internal databases and systems, which could have potentially led to more significant data leaks and compromised services.',
    'date': '2024-04-09',
    'labels': ['layoffs'],
    'sentiment': -0.9946702122688293,
    'liquidity': np.float64(-10.8)},
   {'ticker': 'microsoft',
    'title': 'Microsoft Bing to gain more personalized answers, support for ...',
    'url': 'https://techcrunch.com/2023/09/22/microsoft-bing-to-gain-more-personalized-answers-support-for-dalle-e-3-and-watermarked-ai-images/',
    'summary': "Microsoft's Bing search engine is set to receive a boost in AI capabilities, including support for OpenAI's DALLE-E 3 model, more personalized answers, and watermarked AI-generated images, as part of a broader effort to integrate AI into its Windows and Microsoft 365 products.",
    'date': '2023-09-22',
    'labels': [],
    'sentiment': 0.9987637996673584,
    'liquidity': np.float64(-29.7)},
   {'ticker': 'microsoft',
    'title': 'Microsoft confirms breach after hackers publish source code',
    'url': 'https://techcrunch.com/2022/03/23/microsoft-lapsus-hack-source-code/',
    'summary': "Microsoft has confirmed a breach by the Lapsus$ hacking group, which compromised a single employee's account and stole partial source code from Bing, Bing Maps, and Cortana, with no customer data or code affected, and the company's cybersecurity teams quickly intervened to limit the impact.",
    'date': '2022-03-23',
    'labels': [],
    'sentiment': -0.9994956254959106,
    'liquidity': np.float64(-14.4)},
   {'ticker': 'microsoft',
    'title': 'Microsoft commits $25M to its AI for Accessibility program ...',
    'url': 'https://techcrunch.com/2018/05/07/microsoft-commits-25m-to-its-ai-for-accessibility-program/#!',
    'summary': 'Microsoft has committed $25 million to its AI for Accessibility program over the next five years, aiming to provide seed grants to developers, universities, and NGOs to create solutions that assist people with disabilities and mental health conditions, with the goal of increasing accessibility and opportunities for this community.',
    'date': '2018-05-07',
    'labels': [],
    'sentiment': 0.8279069066047668,
    'liquidity': np.float64(-26.5)},
   {'ticker': 'microsoft',
    'title': 'Facebook, Amazon, Google, IBM and Microsoft come together to ...',
    'url': 'https://techcrunch.com/2016/09/28/facebook-amazon-google-ibm-and-microsoft-come-together-to-create-historic-partnership-on-ai/',
    'summary': "Here is a concise news summary:\n\nThe world's largest technology companies, including Facebook, Amazon, Google, IBM, and Microsoft, have formed the Partnership on AI, a collaborative effort to conduct research and promote best practices in artificial intelligence, aiming to self-regulate the industry and ensure responsible development of AI technologies.",
    'date': '2016-09-28',
    'labels': [],
    'sentiment': 0.9988577365875244,
    'liquidity': np.float64(-4.2)},
   {'ticker': 'microsoft',
    'title': 'Microsoft Takes .NET Open Source And Cross-Platform | TechCrunch',
    'url': 'https://techcrunch.com/2014/11/12/microsoft-takes-net-open-source-and-cross-platform/',
    'summary': 'Microsoft has announced plans to take its .NET framework open source and cross-platform, allowing developers to build applications on Mac and Linux, in a move that could potentially impact 6 million developers currently using the framework, with no financial details disclosed.',
    'date': '2014-11-12',
    'labels': [],
    'sentiment': -0.9946452379226685,
    'liquidity': np.float64(19.7)},
   {'ticker': 'microsoft',
    'title': 'Rumor: Microsoft\'s "Metro" Design Language May Be Renamed ...',
    'url': 'https://techcrunch.com/2012/08/09/metro-ms-windows8/',
    'summary': 'Microsoft has reportedly dropped its "Metro" design language, replacing it with "Windows 8", citing a desire to transition from industry jargon to a broader consumer dialogue, although the change may also be due to potential legal issues with German retailer Metro AG.',
    'date': '2012-08-09',
    'labels': ['legal issues'],
    'sentiment': -0.9992689490318298,
    'liquidity': np.float64(-1.6)},
   {'ticker': 'microsoft',
    'title': 'PatentMonkey: Microsoft Surface Not Just a Touch Screen ...',
    'url': 'https://techcrunch.com/2007/05/30/patentmonkey-microsoft-surface-not-just-a-touch-screen/',
    'summary': "Microsoft's Surface technology, which uses IR sensors to detect objects on its surface, has been patented, allowing for additional layers of input and direction beyond traditional touch screens, with the company filing a patent application in June 2005.\n\n**Key entities:**\n\n* Microsoft\n* Popular Mechanics\n* Sequoia\n* Aspora\n* U.S. Navy\n* Taiwan\n* Huawei\n* SMIC\n* Alexa von Tobel\n* Google\n* Scale AI\n* Precursor Ventures\n* NEA\n* Index Ventures\n* Underscore VC\n\n**Financial impacts and numbers:**\n\n* Aspora receives $50M from Sequoia\n* No specific financial numbers mentioned for Microsoft's Surface technology or patent application",
    'date': '2007-05-30',
    'labels': [],
    'sentiment': -0.9958595633506775,
    'liquidity': np.float64(-10.3)}]},
 {'apple': [{'ticker': 'apple',
    'title': "Orb, a new app by Ookla's ex-CEO, offers a broader picture of your ...",
    'url': 'https://techcrunch.com/2025/04/30/former-ookla-ceos-new-app-aims-to-measure-internet-reliablity-along-with-speed/',
    'summary': 'Doug Suttles, former CEO of Ookla, has launched Orb, a new app that measures internet connection stability by combining latency, packet loss, jitter, and speed, aiming to provide a more comprehensive picture of internet performance, with a score indicating stability.',
    'date': '2025-04-30',
    'labels': [],
    'sentiment': 0.9858829379081726,
    'liquidity': np.float64(71.4)},
   {'ticker': 'apple',
    'title': 'Apple rolls out Priority Notifications as Apple Intelligence expands to ...',
    'url': 'https://techcrunch.com/2025/03/31/apple-rolls-out-priority-notifications-as-apple-intelligence-expands-to-eu/',
    'summary': 'Apple has rolled out Priority Notifications as part of its Apple Intelligence suite, allowing users to prioritize time-sensitive notifications, alongside other updates to Image Playground and the Mac, as the AI-powered features expand to the EU and Apple Vision Pro headset in the US, with the changes available with the release of iOS 18.4, iPadOS 18.4, and macOS Sequoia 15.4.',
    'date': '2025-03-31',
    'labels': [],
    'sentiment': 0.9057695269584656,
    'liquidity': np.float64(29.2)},
   {'ticker': 'apple',
    'title': "Here's how OpenAI uses your data when you use ChatGPT through ...",
    'url': 'https://techcrunch.com/2024/12/11/heres-how-openai-uses-your-data-when-you-use-chatgpt-through-apples-integrations/',
    'summary': "OpenAI, the company behind the popular ChatGPT AI model, has clarified how user data is handled when used through Apple's integrations, stating that it does not receive IP addresses or store requests when used without an account, but does process data in accordance with its privacy policy when users log in or continue conversations on the app or website.",
    'date': '2024-12-11',
    'labels': [],
    'sentiment': 0.9352881908416748,
    'liquidity': np.float64(-5.2)},
   {'ticker': 'apple',
    'title': 'Apple announces new accessibility features for iPhone and iPad ...',
    'url': 'https://techcrunch.com/2024/05/15/apple-accessibility-features-2024/',
    'summary': 'Apple has announced a range of new accessibility features for iPhone and iPad users, including built-in eye-tracking technology, custom voice shortcuts, and improved speech recognition, designed to cater to a diverse range of user needs and abilities, with no additional hardware or accessories required.',
    'date': '2024-05-15',
    'labels': [],
    'sentiment': -0.9653823375701904,
    'liquidity': np.float64(-20.3)},
   {'ticker': 'apple',
    'title': "Despite complaints, Apple hasn't yet removed an obviously fake app ...",
    'url': 'https://techcrunch.com/2024/04/26/apple-hasnt-yet-removed-obviously-fake-app-pretending-to-be-rockauto/',
    'summary': "Apple's App Store has failed to remove a fake app masquerading as RockAuto, an auto parts dealer, despite numerous complaints, leaving customers vulnerable to potential scams and financial losses, with RockAuto's co-founder and president Jim Taylor expressing frustration over the lack of action from Apple.",
    'date': '2024-04-26',
    'labels': [],
    'sentiment': -0.9993467926979065,
    'liquidity': np.float64(37.2)},
   {'ticker': 'apple',
    'title': "Apple spotted developing a 'multiview' feature for watching sports on ...",
    'url': 'https://techcrunch.com/2023/03/29/apple-spotted-developing-a-multiview-feature-for-watching-sports-on-apple-tv/',
    'summary': 'Apple is reportedly developing a "multiview" feature for its Apple TV platform, allowing users to watch up to four streams of sports content simultaneously, following a similar feature launched by YouTube TV earlier this month.\n\n**Key entities:**\n\n* Apple\n* YouTube TV\n* Major League Baseball\n\n**Financial impacts and numbers:**\n\n* No specific financial numbers mentioned in the article.',
    'date': '2023-03-29',
    'labels': [],
    'sentiment': -0.9965080618858337,
    'liquidity': np.float64(-4.1)},
   {'ticker': 'apple',
    'title': "Apple's App Tracking Transparency coming in early spring",
    'url': 'https://techcrunch.com/2021/01/27/apple-app-tracking-transparency/',
    'summary': 'Apple has announced that its App Tracking Transparency feature, which requires app developers to ask users for permission to track their IDFA identifier for cross-property ad targeting purposes, will be enabled by default and arrive in early spring on iOS, iPadOS, and tvOS, with users able to view and manage tracking permissions under Settings.',
    'date': '2021-01-28',
    'labels': [],
    'sentiment': -0.9184266924858093,
    'liquidity': np.float64(-12.5)},
   {'ticker': 'apple',
    'title': "Platform unity and App Store push keep Apple's gaming lead",
    'url': 'https://techcrunch.com/2018/03/21/mobile-gaming-is-having-a-moment-and-apple-has-the-reins/',
    'summary': "Apple's dominance in the tech news cycle continued this week as the launch of Fortnite and PlayerUnknown's Battlegrounds on mobile devices, specifically iOS, stole the spotlight at the 2018 Game Developers Conference, marking a significant shift in the gaming industry's platform discussion and potentially changing the psychology around mobile gaming.",
    'date': '2018-03-21',
    'labels': [],
    'sentiment': 0.9975703358650208,
    'liquidity': np.float64(74.5)},
   {'ticker': 'apple',
    'title': "Apple's Latest Crackdown: Apps Pulling The Advertising Identifier ...",
    'url': 'https://techcrunch.com/2014/02/03/apples-latest-crackdown-apps-pulling-the-advertising-identifier-but-not-showing-ads-are-being-rejected-from-app-store/',
    'summary': 'Apple has begun rejecting mobile applications from its App Store that retrieve the Identifier for Advertisers (IDFA) but do not display ads, enforcing a clause that prohibits the use of IDFA for tracking purposes without displaying ads, in a move aimed at promoting transparency and user privacy.',
    'date': '2014-02-03',
    'labels': [],
    'sentiment': -0.9633257389068604,
    'liquidity': np.float64(-30.3)},
   {'ticker': 'apple',
    'title': 'Apple Releases iOS 7.0.2 With Fixes For Lock Screen Bypass ...',
    'url': 'https://techcrunch.com/2013/09/26/apple-releases-ios-7-0-2-with-fix-for-lock-screen-bypass/',
    'summary': 'Apple has released iOS 7.0.2, addressing a lock screen bypass vulnerability that allowed users to access photos and social sharing options, while also reintroducing a Greek keyboard option for passcode entry, with the update available for download over-the-air.\n\nKey entities:\n\n* Apple\n* TechCrunch\n* Aspora\n* Sequoia\n* U.S. Navy\n* Huawei\n* SMIC\n* Alexa von Tobel\n* Precursor Ventures\n* NEA\n* Index Ventures\n* Underscore VC\n* 23andMe\n* Scale AI\n* Amazon\n* Google\n* Meta\n* Microsoft\n\nFinancial impacts and numbers:\n\n* Aspora receives $50M from Sequoia to build remittance and banking solutions for Indian diaspora\n* No specific financial numbers mentioned for other entities or transactions',
    'date': '2013-09-26',
    'labels': [],
    'sentiment': -0.9958524703979492,
    'liquidity': np.float64(-34.4)}]},
 {'amazon': [{'ticker': 'amazon',
    'title': 'Solar power magnate Gautam Adani and others indicted over ...',
    'url': 'https://techcrunch.com/2024/11/20/solar-power-magnate-gautam-adani-and-others-indicted-over-alleged-250m-bribery-scheme/',
    'summary': 'Billionaire Gautam Adani and several executives at his Indian conglomerate, Adani Group, have been indicted by the US Department of Justice and Securities and Exchange Commission (SEC) for allegedly paying over $250 million in bribes to Indian officials to secure contracts for a 12-gigawatt solar power project, with the SEC also charging the Adanis and others for violating federal antifraud laws in raising $175 million from US investors.',
    'date': '2024-11-20',
    'labels': [],
    'sentiment': -0.992850661277771,
    'liquidity': np.float64(-20.8)},
   {'ticker': 'amazon',
    'title': 'Amazon Music follows Spotify with an AI playlist generator of its own ...',
    'url': 'https://techcrunch.com/2024/04/16/amazon-music-follows-spotify-with-an-ai-playlist-generator-of-its-own-maestro/',
    'summary': "Amazon Music has launched Maestro, an AI playlist generator, in beta, allowing US customers to create playlists using spoken or written prompts, including emojis, with the goal of matching songs to user input, following in the footsteps of Spotify's similar feature.",
    'date': '2024-04-16',
    'labels': [],
    'sentiment': -0.9960048794746399,
    'liquidity': np.float64(-0.4)},
   {'ticker': 'amazon',
    'title': 'Amazon exec confirms corporate hiring freeze through end of year ...',
    'url': 'https://techcrunch.com/2022/11/03/amazon-exec-confirms-corporate-hiring-freeze-through-end-of-year/',
    'summary': 'Amazon has confirmed a corporate hiring freeze through the end of the year, citing an "unusual macro-economic environment" and a desire to balance hiring and investments, with Senior Vice President of People Experience and Technology Beth Galetti noting that the company plans to add a "meaningful number of people" next year.',
    'date': '2022-11-03',
    'labels': [],
    'sentiment': 0.7585660815238953,
    'liquidity': np.float64(-32.2)},
   {'ticker': 'amazon',
    'title': 'Judge orders Amazon to turn over Echo recordings in double ...',
    'url': 'https://techcrunch.com/2018/11/14/amazon-echo-recordings-judge-murder-case/',
    'summary': 'Here is a concise news summary:\n\nA New Hampshire judge has ordered Amazon to turn over two days of Amazon Echo recordings in a double murder case, where prosecutors believe the recordings may yield further clues to the killer, with Amazon directed to provide any audio recordings capturing the attack and information identifying cellular devices linked to the smart speaker during the time period.',
    'date': '2018-11-14',
    'labels': ['legal issues'],
    'sentiment': -0.9404541254043579,
    'liquidity': np.float64(13.5)},
   {'ticker': 'amazon',
    'title': 'Amazon Echo and Alexa arrive in Mexico | TechCrunch',
    'url': 'https://techcrunch.com/2018/11/07/amazon-echo-and-alexa-arrive-in-mexico/',
    'summary': "Amazon has expanded its Alexa-powered Echo devices to Mexico, marking the smart assistant's availability in around 40 countries, with pre-orders starting today and shipping set to begin next week, as the company continues to globalize its AI technology.",
    'date': '2018-11-07',
    'labels': [],
    'sentiment': 0.9959598183631897,
    'liquidity': np.float64(14.0)},
   {'ticker': 'amazon',
    'title': 'Alexa can now reserve conference rooms | TechCrunch',
    'url': 'https://techcrunch.com/2018/10/10/alexa-can-now-reserve-conference-rooms/',
    'summary': "Amazon has introduced a new feature to its Alexa for Business platform, allowing businesses to book conference rooms using voice commands, expanding the smart assistant's capabilities beyond the home and marking a significant step in Amazon's efforts to put Alexa to work in the enterprise.",
    'date': '2018-10-10',
    'labels': [],
    'sentiment': 0.9990642666816711,
    'liquidity': np.float64(32.8)},
   {'ticker': 'amazon',
    'title': "Amazon's a la carte TV service, Amazon Channels, adds CBS All ...",
    'url': 'https://techcrunch.com/2018/01/05/amazons-a-la-carte-tv-service-amazon-channels-adds-cbs-all-access/',
    'summary': "Amazon's a la carte TV service, Amazon Channels, has added CBS All Access to its lineup, offering a commercial-free subscription for $9.99/month, with a lower-cost option with ads expected to follow in the coming months.",
    'date': '2018-01-05',
    'labels': [],
    'sentiment': -0.9893101453781128,
    'liquidity': np.float64(18.4)},
   {'ticker': 'amazon',
    'title': 'Amazon Kindle Owners Are "Borrowing" Nearly 300,000 Electronic ...',
    'url': 'https://techcrunch.com/2012/01/12/amazon-kindle-owners-are-borrowing-nearly-300000-electronic-books-a-month/',
    'summary': "Amazon's Kindle Owners' Lending Library has seen significant growth, with nearly 300,000 e-book borrows in December alone, and the company has responded by increasing the January KDP Select fund to $700,000, benefiting authors who have earned up to $70,000 in royalties and seen a 449% month-over-month increase in total royalties.",
    'date': '2012-01-12',
    'labels': [],
    'sentiment': 0.9979355335235596,
    'liquidity': np.float64(36.6)},
   {'ticker': 'amazon',
    'title': 'Amazon Will Soon Allow Kindle Users To Lend E-Books | TechCrunch',
    'url': 'https://techcrunch.com/2010/10/22/amazon-will-soon-allow-kindle-users-to-lend-e-books/',
    'summary': 'Amazon is set to introduce a new feature on its Kindle e-readers, allowing users to lend e-books to friends and family for a 14-day period, while also expanding its Kindle newspapers and magazines feature to its iOS apps, with plans to roll it out to other platforms soon.',
    'date': '2010-10-22',
    'labels': [],
    'sentiment': 0.9929620623588562,
    'liquidity': np.float64(-31.2)}]},
 {'_id': ObjectId('685030b0333fbb2aba57e63e')}]